In [7]:
!pip install catboost

In [2]:
# ============================================================
# 단계 1: 데이터 적재 및 전처리 파이프라인 구축 (개선 버전)
# ============================================================

import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, RobustScaler
import warnings
warnings.filterwarnings('ignore')

print("=" * 70)
print("단계 1: 데이터 적재 및 전처리 파이프라인 (개선 버전)")
print("=" * 70)

# ============================================================
# 1.1 데이터 로드 및 기본 정보
# ============================================================
df = pd.read_csv('train.csv')

print(f"\n[데이터 기본 정보]")
print(f"데이터 크기: {df.shape}")
print(f"컬럼 수: {len(df.columns)}")
print(f"샘플 수: {len(df):,}개")

# 데이터 타입 확인
print(f"\n[데이터 타입 분포]")
print(df.dtypes.value_counts())

# 결측치 확인
print(f"\n[결측치 분석]")
missing_count = df.isna().sum().sum()
print(f"전체 결측치: {missing_count:,}개")

if missing_count > 0:
    missing_cols = df.columns[df.isna().any()].tolist()
    print(f"결측치가 있는 컬럼: {len(missing_cols)}개")
    for col in missing_cols[:5]:  # 상위 5개만 출력
        print(f"  - {col}: {df[col].isna().sum()}개 ({df[col].isna().mean()*100:.2f}%)")

# ============================================================
# 1.2 라벨 분포 및 클래스 불균형 분석
# ============================================================
print(f"\n[라벨 분포 분석]")
label_counts = df['label'].value_counts()
label_ratio = df['label'].value_counts(normalize=True)

print("클래스별 샘플 수:")
for label, count in label_counts.items():
    ratio = label_ratio[label] * 100
    print(f"  - 클래스 {label}: {count:,}개 ({ratio:.2f}%)")

# 클래스 불균형 비율 계산
imbalance_ratio = label_counts.max() / label_counts.min()
print(f"\n클래스 불균형 비율: {imbalance_ratio:.2f}:1")

if imbalance_ratio > 1.5:
    print("⚠️  클래스 불균형 감지 → class_weight='balanced' 사용 권장")
else:
    print("✓ 클래스 균형이 양호합니다")

# ============================================================
# 1.3 컬럼 그룹 정의
# ============================================================
id_col = 'SMILES'
label_col = 'label'

# Fingerprint 컬럼 (ecfp, fcfp, ptfp)
fp_cols = [col for col in df.columns if col.startswith(('ecfp_', 'fcfp_', 'ptfp_'))]

# 물성 descriptor 컬럼
desc_cols = ['MolWt', 'clogp', 'sa_score', 'qed']

print(f"\n[컬럼 그룹 정의]")
print(f"Fingerprint 컬럼: {len(fp_cols)}개")
print(f"  - ECFP: {len([c for c in fp_cols if c.startswith('ecfp_')])}개")
print(f"  - FCFP: {len([c for c in fp_cols if c.startswith('fcfp_')])}개")
print(f"  - PTFP: {len([c for c in fp_cols if c.startswith('ptfp_')])}개")
print(f"물성 Descriptor: {desc_cols}")

# ============================================================
# 1.4 Feature 통계 분석
# ============================================================
print(f"\n[물성 Descriptor 통계]")
desc_stats = df[desc_cols].describe().T
print(desc_stats[['mean', 'std', 'min', 'max']])

# 이상치 감지
print(f"\n[이상치 분석 (IQR 기반)]")
for col in desc_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    outliers = df[(df[col] < Q1 - 1.5*IQR) | (df[col] > Q3 + 1.5*IQR)][col]
    if len(outliers) > 0:
        print(f"  - {col}: {len(outliers)}개 ({len(outliers)/len(df)*100:.2f}%)")

# Fingerprint 희소성 분석
print(f"\n[Fingerprint 희소성 분석]")
fp_data = df[fp_cols]
sparsity = (fp_data == 0).sum().sum() / (fp_data.shape[0] * fp_data.shape[1])
print(f"희소성: {sparsity*100:.2f}% (0의 비율)")

# ============================================================
# 1.5 Feature Importance 기반 선택 (선택적)
# ============================================================
# Feature importance 파일이 있다면 로드
try:
    feature_importance = pd.read_csv('feature_importance_cv.csv')
    print(f"\n[Feature Importance 로드]")
    print(f"총 피처: {len(feature_importance)}개")

    # 상위 피처 선택 (임계값: importance_mean > 50)
    importance_threshold = 50
    selected_by_importance = feature_importance[
        feature_importance['importance_mean'] > importance_threshold
    ]['feature'].tolist()

    # Descriptor는 항상 포함
    selected_features = list(set(selected_by_importance + desc_cols))

    # Fingerprint 중 선택된 것만 필터링
    fp_cols_selected = [col for col in selected_features if col in fp_cols]

    print(f"\n선택된 피처 ({importance_threshold} 이상):")
    print(f"  - Fingerprint: {len(fp_cols_selected)}개")
    print(f"  - Descriptor: {len([c for c in selected_features if c in desc_cols])}개")
    print(f"  - 총: {len(selected_features)}개")

    # 피처 선택 적용
    use_feature_selection = True

except FileNotFoundError:
    print(f"\n[Feature Importance 파일 없음 - 전체 피처 사용]")
    selected_features = fp_cols + desc_cols
    fp_cols_selected = fp_cols
    use_feature_selection = False

# ============================================================
# 1.6 X, y 분리
# ============================================================
if use_feature_selection:
    X = df[selected_features]
else:
    X = df.drop(columns=[label_col])

y = df[label_col].astype(int)

print(f"\n[학습 데이터 준비]")
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

# ============================================================
# 1.7 전처리 파이프라인 구성 (개선)
# ============================================================
# Fingerprint 전처리: 0으로 대치
fp_transformer = SimpleImputer(strategy='constant', fill_value=0)

# Descriptor 전처리: RobustScaler 사용 (이상치에 강건)
desc_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', RobustScaler())  # StandardScaler 대신 RobustScaler 사용
    # RobustScaler는 중앙값과 IQR을 사용하여 이상치에 더 강건함
])

# 사용할 fingerprint 컬럼 결정
fp_cols_to_use = fp_cols_selected if use_feature_selection else fp_cols

preprocessor = ColumnTransformer(
    transformers=[
        ('fp', fp_transformer, fp_cols_to_use),
        ('desc', desc_transformer, desc_cols)
    ],
    remainder='drop'
)

print(f"\n[전처리 파이프라인 구축 완료]")
print(f"  - Fingerprint: 결측치 → 0 대치 ({len(fp_cols_to_use)}개 컬럼)")
print(f"  - Descriptor: 결측치 → 중앙값 대치 + RobustScaler (이상치 강건)")

# ============================================================
# 1.8 교차검증 설정
# ============================================================
N_SPLITS = 5
RANDOM_STATE = 42

skf = StratifiedKFold(
    n_splits=N_SPLITS,
    shuffle=True,
    random_state=RANDOM_STATE
)

print(f"  - 교차검증: {N_SPLITS}-Fold Stratified")

# ============================================================
# 1.9 샘플 변환 테스트 및 검증
# ============================================================
print(f"\n[전처리 변환 테스트]")

for fold, (tr_idx, va_idx) in enumerate(skf.split(X, y), 1):
    X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
    y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]

    # 전처리 적용
    Xt_tr = preprocessor.fit_transform(X_tr)
    Xt_va = preprocessor.transform(X_va)

    # Fold 정보
    print(f"\nFold {fold}:")
    print(f"  - 학습 데이터: {Xt_tr.shape}")
    print(f"  - 검증 데이터: {Xt_va.shape}")
    print(f"  - 학습 라벨 분포: Class 0={sum(y_tr==0)}, Class 1={sum(y_tr==1)}")
    print(f"  - 검증 라벨 분포: Class 0={sum(y_va==0)}, Class 1={sum(y_va==1)}")

    # 변환 후 통계
    print(f"  - 학습 데이터 통계: mean={Xt_tr.mean():.4f}, std={Xt_tr.std():.4f}")
    print(f"  - 검증 데이터 통계: mean={Xt_va.mean():.4f}, std={Xt_va.std():.4f}")

    break  # 첫 fold만 확인

# ============================================================
# 1.10 전처리 파이프라인 및 설정 저장
# ============================================================
# 전체 데이터로 전처리기 학습 (나중에 테스트 데이터에 사용)
preprocessor_full = ColumnTransformer(
    transformers=[
        ('fp', fp_transformer, fp_cols_to_use),
        ('desc', desc_transformer, desc_cols)
    ],
    remainder='drop'
)

preprocessor_full.fit(X)

print(f"\n[전처리 파이프라인 저장]")
print(f"✓ 전체 데이터로 전처리기 학습 완료")

# 설정 저장
config = {
    'n_splits': N_SPLITS,
    'random_state': RANDOM_STATE,
    'fp_cols': fp_cols_to_use,
    'desc_cols': desc_cols,
    'use_feature_selection': use_feature_selection,
    'n_features': len(fp_cols_to_use) + len(desc_cols),
    'class_imbalance_ratio': imbalance_ratio
}

print(f"\n[설정 요약]")
for key, value in config.items():
    if isinstance(value, list):
        print(f"  - {key}: {len(value)}개")
    else:
        print(f"  - {key}: {value}")

print("\n" + "=" * 70)
print("✓ 단계 1 완료 - 전처리 파이프라인 구축 및 검증 완료")
print("=" * 70)

# 다음 단계를 위한 변수 저장
globals().update({
    'preprocessor': preprocessor_full,
    'X_data': X,
    'y_data': y,
    'skf': skf,
    'config': config
})


단계 1: 데이터 적재 및 전처리 파이프라인 (개선 버전)

[데이터 기본 정보]
데이터 크기: (8349, 3078)
컬럼 수: 3078
샘플 수: 8,349개

[데이터 타입 분포]
int64      3073
float64       4
object        1
Name: count, dtype: int64

[결측치 분석]
전체 결측치: 0개

[라벨 분포 분석]
클래스별 샘플 수:
  - 클래스 1: 4,542개 (54.40%)
  - 클래스 0: 3,807개 (45.60%)

클래스 불균형 비율: 1.19:1
✓ 클래스 균형이 양호합니다

[컬럼 그룹 정의]
Fingerprint 컬럼: 3072개
  - ECFP: 1024개
  - FCFP: 1024개
  - PTFP: 1024개
물성 Descriptor: ['MolWt', 'clogp', 'sa_score', 'qed']

[물성 Descriptor 통계]
                mean        std        min          max
MolWt     443.248753  88.876374  94.117000  1242.488000
clogp       3.794829   1.379045  -4.048930     9.429480
sa_score    3.187613   0.727768   1.282432     7.309297
qed         0.559151   0.185664   0.024365     0.947494

[이상치 분석 (IQR 기반)]
  - MolWt: 130개 (1.56%)
  - clogp: 137개 (1.64%)
  - sa_score: 158개 (1.89%)

[Fingerprint 희소성 분석]
희소성: 82.59% (0의 비율)

[Feature Importance 로드]
총 피처: 3076개

선택된 피처 (50 이상):
  - Fingerprint: 281개
  - Descriptor: 4개
  - 총: 285개

[학습 데이터 준

In [9]:
# ============================================================
# 단계 2: 베이스라인 모델 학습 (최종 개선 버전)
# ============================================================

from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import (
    f1_score, classification_report, confusion_matrix,
    roc_auc_score, precision_recall_curve, roc_curve,
    precision_score, recall_score
)
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

print("\n" + "=" * 70)
print("단계 2: 베이스라인 모델 학습 (최종 개선 버전)")
print("=" * 70)
print("\n[적용된 개선사항]")
print("  1. ✓ Threshold 재최적화 (OOF 기반 자동 탐색)")
print("  2. ✓ XGBoost 중심 Ensemble (50% 가중치)")
print("  3. ✓ Class Weighting (FP 페널티 증가)")
print("  4. ✓ Sample Weighting (Low Confidence 샘플 집중)")

# ============================================================
# 2.1 Feature Selection (기존 코드 유지)
# ============================================================
print(f"\n{'='*70}")
print("Feature Selection")
print(f"{'='*70}")

try:
    feature_importance = pd.read_csv('feature_importance_cv.csv')
    importance_threshold = 30

    selected_features_from_importance = feature_importance[
        feature_importance['importance_mean'] > importance_threshold
    ]['feature'].tolist()

    desc_cols = ['MolWt', 'clogp', 'sa_score', 'qed']
    selected_features = list(set(selected_features_from_importance + desc_cols))
    available_features = [f for f in selected_features if f in X_data.columns]

    X_selected = X_data[available_features]

    print(f"\n[Feature Selection 결과]")
    print(f"  원본 피처: {X_data.shape[1]}개")
    print(f"  실제 사용: {len(available_features)}개")

    feature_selection_applied = True
    selected_features = available_features

except:
    X_selected = X_data.copy()
    selected_features = X_data.columns.tolist()
    feature_selection_applied = False
    print(f"\n⚠️  전체 피처 사용: {X_selected.shape[1]}개")

# ============================================================
# 2.2 초기 설정
# ============================================================
RANDOM_STATE = 42

# 1차 학습용 (Threshold 최적화를 위한)
results_stage1 = {
    'lgbm': {'oof_probabilities': np.zeros(len(X_selected))},
    'xgb': {'oof_probabilities': np.zeros(len(X_selected))},
    'catboost': {'oof_probabilities': np.zeros(len(X_selected))},
    'ensemble': {'oof_probabilities': np.zeros(len(X_selected))}
}

# 최종 결과 저장용
results_final = {
    'lgbm': {'f1_scores': [], 'auc_scores': [], 'models': [],
             'oof_predictions': np.zeros(len(X_selected)),
             'oof_probabilities': np.zeros(len(X_selected))},
    'xgb': {'f1_scores': [], 'auc_scores': [], 'models': [],
            'oof_predictions': np.zeros(len(X_selected)),
            'oof_probabilities': np.zeros(len(X_selected))},
    'catboost': {'f1_scores': [], 'auc_scores': [], 'models': [],
                 'oof_predictions': np.zeros(len(X_selected)),
                 'oof_probabilities': np.zeros(len(X_selected))},
    'ensemble': {'oof_probabilities': np.zeros(len(X_selected)),
                 'oof_predictions': np.zeros(len(X_selected))},
    'fold_details': []
}

print(f"\n[모델 설정]")
print(f"  Random State: {RANDOM_STATE}")
print(f"  전체 샘플 수: {len(X_selected):,}개")
print(f"  사용 피처 수: {X_selected.shape[1]}개")

# ============================================================
# 2.3 1차 학습: Threshold 최적화를 위한 확률 예측
# ============================================================
print(f"\n{'='*70}")
print("1단계: Threshold 최적화를 위한 OOF 확률 수집")
print(f"{'='*70}")

for fold, (tr_idx, va_idx) in enumerate(skf.split(X_selected, y_data), 1):
    print(f"\rFold {fold}/5 처리 중...", end='')

    X_tr, X_va = X_selected.iloc[tr_idx], X_selected.iloc[va_idx]
    y_tr, y_va = y_data.iloc[tr_idx], y_data.iloc[va_idx]

    Xt_tr = preprocessor.fit_transform(X_tr)
    Xt_va = preprocessor.transform(X_va)

    # LightGBM
    lgbm_model = LGBMClassifier(
        n_estimators=1000, learning_rate=0.03, max_depth=8,
        num_leaves=63, min_child_samples=30, subsample=0.8,
        colsample_bytree=0.8, reg_alpha=0.3, reg_lambda=0.3,
        class_weight={0: 1.5, 1: 1.0},  # Class 0(독성)에 가중치
        random_state=RANDOM_STATE, n_jobs=-1, verbose=-1
    )
    lgbm_model.fit(Xt_tr, y_tr, eval_set=[(Xt_va, y_va)],
                   callbacks=[lgb.early_stopping(stopping_rounds=100, verbose=False)])
    results_stage1['lgbm']['oof_probabilities'][va_idx] = lgbm_model.predict_proba(Xt_va)[:, 1]

    # XGBoost
    xgb_model = XGBClassifier(
        n_estimators=1000, learning_rate=0.03, max_depth=7,
        min_child_weight=3, subsample=0.8, colsample_bytree=0.8,
        gamma=0.1, reg_alpha=0.3, reg_lambda=0.3,
        scale_pos_weight=0.67,  # Class 1이 많으므로 Class 0 강화
        random_state=RANDOM_STATE, n_jobs=-1,
        early_stopping_rounds=100, eval_metric='logloss', verbosity=0
    )
    xgb_model.fit(Xt_tr, y_tr, eval_set=[(Xt_va, y_va)], verbose=False)
    results_stage1['xgb']['oof_probabilities'][va_idx] = xgb_model.predict_proba(Xt_va)[:, 1]

    # CatBoost
    cat_model = CatBoostClassifier(
        iterations=1000, learning_rate=0.03, depth=7,
        l2_leaf_reg=3, class_weights=[1.5, 1.0],  # Class 0 강화
        random_seed=RANDOM_STATE, verbose=0,
        early_stopping_rounds=100
    )
    cat_model.fit(Xt_tr, y_tr, eval_set=(Xt_va, y_va), verbose=False)
    results_stage1['catboost']['oof_probabilities'][va_idx] = cat_model.predict_proba(Xt_va)[:, 1]

print(f"\r✓ 1단계 완료: 5-Fold OOF 확률 수집 완료")

# XGBoost 중심 Ensemble (50% 가중치)
results_stage1['ensemble']['oof_probabilities'] = (
    0.25 * results_stage1['lgbm']['oof_probabilities'] +
    0.50 * results_stage1['xgb']['oof_probabilities'] +  # XGBoost 증가
    0.25 * results_stage1['catboost']['oof_probabilities']
)

# ============================================================
# 2.4 Threshold 최적화 (FPR 고려)
# ============================================================
print(f"\n{'='*70}")
print("2단계: Threshold 최적화 (FPR 페널티 적용)")
print(f"{'='*70}")

def optimize_threshold_with_fpr(y_true, y_pred_proba, max_fpr=0.25):
    """FPR 제약 하에서 F1 Score 최적화"""
    thresholds = np.arange(0.1, 0.9, 0.005)
    best_f1 = 0
    best_threshold = 0.5
    best_fpr = 1.0

    results_list = []

    for thresh in thresholds:
        y_pred = (y_pred_proba >= thresh).astype(int)

        # F1 Score
        f1 = f1_score(y_true, y_pred)

        # FPR 계산
        cm = confusion_matrix(y_true, y_pred)
        tn, fp, fn, tp = cm.ravel()
        fpr = fp / (fp + tn) if (fp + tn) > 0 else 0

        results_list.append({
            'threshold': thresh,
            'f1': f1,
            'fpr': fpr,
            'precision': tp / (tp + fp) if (tp + fp) > 0 else 0,
            'recall': tp / (tp + fn) if (tp + fn) > 0 else 0
        })

        # FPR 제약 조건 만족하면서 F1 최대화
        if fpr <= max_fpr and f1 > best_f1:
            best_f1 = f1
            best_threshold = thresh
            best_fpr = fpr

    return best_threshold, best_f1, best_fpr, results_list

# Threshold 탐색
print(f"\n[Threshold 탐색]")
print(f"  제약 조건: FPR ≤ 25%")

optimal_threshold, optimal_f1, optimal_fpr, threshold_results = optimize_threshold_with_fpr(
    y_data,
    results_stage1['ensemble']['oof_probabilities'],
    max_fpr=0.25
)

print(f"\n  최적 Threshold: {optimal_threshold:.3f}")
print(f"  예상 F1 Score: {optimal_f1:.4f}")
print(f"  예상 FPR: {optimal_fpr:.4f} ({optimal_fpr*100:.2f}%)")

# 기존 threshold와 비교
baseline_threshold = 0.37
baseline_pred = (results_stage1['ensemble']['oof_probabilities'] >= baseline_threshold).astype(int)
baseline_f1 = f1_score(y_data, baseline_pred)
baseline_cm = confusion_matrix(y_data, baseline_pred)
baseline_fpr = baseline_cm[0,1] / (baseline_cm[0,1] + baseline_cm[0,0])

print(f"\n[이전 vs 최적]")
print(f"  이전 (0.37): F1={baseline_f1:.4f}, FPR={baseline_fpr:.4f} ({baseline_fpr*100:.2f}%)")
print(f"  최적 ({optimal_threshold:.3f}): F1={optimal_f1:.4f}, FPR={optimal_fpr:.4f} ({optimal_fpr*100:.2f}%)")
print(f"  개선: F1 {(optimal_f1-baseline_f1)*100:+.2f}%p, FPR {(optimal_fpr-baseline_fpr)*100:+.2f}%p")

# ============================================================
# 2.5 Sample Weighting: Low Confidence 샘플 가중치
# ============================================================
print(f"\n{'='*70}")
print("3단계: Sample Weighting 적용")
print(f"{'='*70}")

# Low Confidence 샘플 식별
confidence = np.abs(results_stage1['ensemble']['oof_probabilities'] - 0.5)
low_conf_mask = confidence < 0.15  # 임계값 조정

sample_weights = np.ones(len(X_selected))
sample_weights[low_conf_mask] = 2.0  # Low Confidence에 2배 가중치

n_low_conf = low_conf_mask.sum()
print(f"\n[Sample Weighting]")
print(f"  Low Confidence 샘플: {n_low_conf}개 ({n_low_conf/len(X_selected)*100:.2f}%)")
print(f"  적용 가중치: 2.0배")

# ============================================================
# 2.6 최종 학습: 개선사항 모두 적용
# ============================================================
print(f"\n{'='*70}")
print("4단계: 최종 3-Model Ensemble 학습")
print(f"{'='*70}")

for fold, (tr_idx, va_idx) in enumerate(skf.split(X_selected, y_data), 1):
    print(f"\n{'─'*70}")
    print(f"📊 Fold {fold}/5")
    print(f"{'─'*70}")

    X_tr, X_va = X_selected.iloc[tr_idx], X_selected.iloc[va_idx]
    y_tr, y_va = y_data.iloc[tr_idx], y_data.iloc[va_idx]
    sample_weight_tr = sample_weights[tr_idx]

    print(f"\n[데이터 분할]")
    print(f"  학습: {len(X_tr):,}개 (Low Conf: {low_conf_mask[tr_idx].sum()}개)")
    print(f"  검증: {len(X_va):,}개")

    Xt_tr = preprocessor.fit_transform(X_tr)
    Xt_va = preprocessor.transform(X_va)

    # ========================================
    # LightGBM (Class Weight + Sample Weight)
    # ========================================
    print(f"\n[1/3] LightGBM...")
    lgbm_model = LGBMClassifier(
        n_estimators=1000, learning_rate=0.03, max_depth=8,
        num_leaves=63, min_child_samples=30, subsample=0.8,
        colsample_bytree=0.8, reg_alpha=0.3, reg_lambda=0.3,
        class_weight={0: 1.5, 1: 1.0},
        random_state=RANDOM_STATE, n_jobs=-1, verbose=-1
    )
    lgbm_model.fit(
        Xt_tr, y_tr,
        sample_weight=sample_weight_tr,
        eval_set=[(Xt_va, y_va)],
        callbacks=[lgb.early_stopping(stopping_rounds=100, verbose=False)]
    )
    lgbm_proba = lgbm_model.predict_proba(Xt_va)[:, 1]
    lgbm_pred = (lgbm_proba >= optimal_threshold).astype(int)

    results_final['lgbm']['models'].append(lgbm_model)
    results_final['lgbm']['oof_probabilities'][va_idx] = lgbm_proba
    results_final['lgbm']['oof_predictions'][va_idx] = lgbm_pred

    lgbm_f1 = f1_score(y_va, lgbm_pred)
    lgbm_auc = roc_auc_score(y_va, lgbm_proba)
    results_final['lgbm']['f1_scores'].append(lgbm_f1)
    results_final['lgbm']['auc_scores'].append(lgbm_auc)

    print(f"  ✓ F1: {lgbm_f1:.4f}, AUC: {lgbm_auc:.4f}, Iter: {lgbm_model.best_iteration_}")

    # ========================================
    # XGBoost (Scale Pos Weight + Sample Weight)
    # ========================================
    print(f"\n[2/3] XGBoost...")
    xgb_model = XGBClassifier(
        n_estimators=1000, learning_rate=0.03, max_depth=7,
        min_child_weight=3, subsample=0.8, colsample_bytree=0.8,
        gamma=0.1, reg_alpha=0.3, reg_lambda=0.3,
        scale_pos_weight=0.67,
        random_state=RANDOM_STATE, n_jobs=-1,
        early_stopping_rounds=100, eval_metric='logloss', verbosity=0
    )
    xgb_model.fit(
        Xt_tr, y_tr,
        sample_weight=sample_weight_tr,
        eval_set=[(Xt_va, y_va)],
        verbose=False
    )
    xgb_proba = xgb_model.predict_proba(Xt_va)[:, 1]
    xgb_pred = (xgb_proba >= optimal_threshold).astype(int)

    results_final['xgb']['models'].append(xgb_model)
    results_final['xgb']['oof_probabilities'][va_idx] = xgb_proba
    results_final['xgb']['oof_predictions'][va_idx] = xgb_pred

    xgb_f1 = f1_score(y_va, xgb_pred)
    xgb_auc = roc_auc_score(y_va, xgb_proba)
    results_final['xgb']['f1_scores'].append(xgb_f1)
    results_final['xgb']['auc_scores'].append(xgb_auc)

    print(f"  ✓ F1: {xgb_f1:.4f}, AUC: {xgb_auc:.4f}, Iter: {xgb_model.best_iteration}")

    # ========================================
    # CatBoost (Class Weights + Sample Weight)
    # ========================================
    print(f"\n[3/3] CatBoost...")
    cat_model = CatBoostClassifier(
        iterations=1000, learning_rate=0.03, depth=7,
        l2_leaf_reg=3, class_weights=[1.5, 1.0],
        random_seed=RANDOM_STATE, verbose=0,
        early_stopping_rounds=100
    )
    cat_model.fit(
        Xt_tr, y_tr,
        sample_weight=sample_weight_tr,
        eval_set=(Xt_va, y_va),
        verbose=False
    )
    cat_proba = cat_model.predict_proba(Xt_va)[:, 1]
    cat_pred = (cat_proba >= optimal_threshold).astype(int)

    results_final['catboost']['models'].append(cat_model)
    results_final['catboost']['oof_probabilities'][va_idx] = cat_proba
    results_final['catboost']['oof_predictions'][va_idx] = cat_pred

    cat_f1 = f1_score(y_va, cat_pred)
    cat_auc = roc_auc_score(y_va, cat_proba)
    results_final['catboost']['f1_scores'].append(cat_f1)
    results_final['catboost']['auc_scores'].append(cat_auc)

    print(f"  ✓ F1: {cat_f1:.4f}, AUC: {cat_auc:.4f}, Iter: {cat_model.best_iteration_}")

    # ========================================
    # Ensemble: XGBoost 중심 (50%)
    # ========================================
    ensemble_proba = (
        0.25 * lgbm_proba +
        0.50 * xgb_proba +  # XGBoost 증가
        0.25 * cat_proba
    )
    ensemble_pred = (ensemble_proba >= optimal_threshold).astype(int)

    results_final['ensemble']['oof_probabilities'][va_idx] = ensemble_proba
    results_final['ensemble']['oof_predictions'][va_idx] = ensemble_pred

    ensemble_f1 = f1_score(y_va, ensemble_pred)
    ensemble_auc = roc_auc_score(y_va, ensemble_proba)

    print(f"\n[Ensemble 성능]")
    print(f"  F1: {ensemble_f1:.4f}, AUC: {ensemble_auc:.4f}")

    # 혼동 행렬
    cm = confusion_matrix(y_va, ensemble_pred)
    tn, fp, fn, tp = cm.ravel()
    fpr = fp / (fp + tn)
    fnr = fn / (fn + tp)

    print(f"\n[혼동 행렬]")
    print(f"  TN: {tn}, FP: {fp}, FN: {fn}, TP: {tp}")
    print(f"  FPR: {fpr:.4f} ({fpr*100:.2f}%)")
    print(f"  FNR: {fnr:.4f} ({fnr*100:.2f}%)")

    # Fold 정보 저장
    results_final['fold_details'].append({
        'fold': fold,
        'lgbm_f1': lgbm_f1, 'xgb_f1': xgb_f1, 'cat_f1': cat_f1,
        'ensemble_f1': ensemble_f1, 'ensemble_auc': ensemble_auc,
        'fpr': fpr, 'fnr': fnr
    })

# ============================================================
# 2.7 최종 결과 분석
# ============================================================
print(f"\n{'='*70}")
print("최종 결과")
print(f"{'='*70}")

ensemble_oof_f1 = f1_score(y_data, results_final['ensemble']['oof_predictions'])
ensemble_oof_auc = roc_auc_score(y_data, results_final['ensemble']['oof_probabilities'])

oof_cm = confusion_matrix(y_data, results_final['ensemble']['oof_predictions'])
tn, fp, fn, tp = oof_cm.ravel()
final_fpr = fp / (fp + tn)
final_fnr = fn / (fn + tp)
final_precision = tp / (tp + fp)
final_recall = tp / (tp + fn)

print(f"\n[Ensemble OOF 성능]")
print(f"  F1 Score:  {ensemble_oof_f1:.4f}")
print(f"  AUC Score: {ensemble_oof_auc:.4f}")
print(f"  Precision: {final_precision:.4f}")
print(f"  Recall:    {final_recall:.4f}")
print(f"  FPR:       {final_fpr:.4f} ({final_fpr*100:.2f}%)")
print(f"  FNR:       {final_fnr:.4f} ({final_fnr*100:.2f}%)")

print(f"\n[OOF 혼동 행렬]")
print(f"              예측: 0    예측: 1")
print(f"  실제: 0  |   {oof_cm[0,0]:4d}      {oof_cm[0,1]:4d}")
print(f"  실제: 1  |   {oof_cm[1,0]:4d}      {oof_cm[1,1]:4d}")

# 개선 효과
baseline_f1 = 0.8290
baseline_fpr_old = 0.3220

print(f"\n[개선 효과]")
print(f"{'지표':<15} {'이전':<10} {'현재':<10} {'개선':<15}")
print(f"{'-'*55}")
print(f"{'F1 Score':<15} {baseline_f1:<10.4f} {ensemble_oof_f1:<10.4f} "
      f"{(ensemble_oof_f1-baseline_f1)*100:+.2f}%p")
print(f"{'FPR':<15} {baseline_fpr_old:<10.4f} {final_fpr:<10.4f} "
      f"{(final_fpr-baseline_fpr_old)*100:+.2f}%p")

# Low Confidence 분석
confidence_final = np.abs(results_final['ensemble']['oof_probabilities'] - 0.5)
low_conf_mask_final = confidence_final < 0.1
n_low_conf_final = low_conf_mask_final.sum()

if n_low_conf_final > 0:
    low_conf_acc_final = (
        results_final['ensemble']['oof_predictions'][low_conf_mask_final] == y_data[low_conf_mask_final]
    ).mean()

    print(f"\n[Low Confidence 샘플]")
    print(f"  이전: 1,029개 (정확도 48.79%)")
    print(f"  현재: {n_low_conf_final}개 (정확도 {low_conf_acc_final:.4f})")
    print(f"  개선: {1029 - n_low_conf_final}개 감소, 정확도 {(low_conf_acc_final - 0.4879)*100:+.2f}%p")

print(f"\n{'='*70}")
print("✓ 최종 개선 모델 학습 완료")
print(f"{'='*70}")

# 전역 변수 저장
globals().update({
    'final_results': results_final,
    'optimal_threshold': optimal_threshold,
    'threshold_results': threshold_results,
    'X_selected': X_selected,
    'selected_features': selected_features
})




단계 2: 베이스라인 모델 학습 (최종 개선 버전)

[적용된 개선사항]
  1. ✓ Threshold 재최적화 (OOF 기반 자동 탐색)
  2. ✓ XGBoost 중심 Ensemble (50% 가중치)
  3. ✓ Class Weighting (FP 페널티 증가)
  4. ✓ Sample Weighting (Low Confidence 샘플 집중)

Feature Selection

[Feature Selection 결과]
  원본 피처: 285개
  실제 사용: 285개

[모델 설정]
  Random State: 42
  전체 샘플 수: 8,349개
  사용 피처 수: 285개

1단계: Threshold 최적화를 위한 OOF 확률 수집
✓ 1단계 완료: 5-Fold OOF 확률 수집 완료

2단계: Threshold 최적화 (FPR 페널티 적용)

[Threshold 탐색]
  제약 조건: FPR ≤ 25%

  최적 Threshold: 0.385
  예상 F1 Score: 0.8314
  예상 FPR: 0.2495 (24.95%)

[이전 vs 최적]
  이전 (0.37): F1=0.8301, FPR=0.2648 (26.48%)
  최적 (0.385): F1=0.8314, FPR=0.2495 (24.95%)
  개선: F1 +0.13%p, FPR -1.52%p

3단계: Sample Weighting 적용

[Sample Weighting]
  Low Confidence 샘플: 1521개 (18.22%)
  적용 가중치: 2.0배

4단계: 최종 3-Model Ensemble 학습

──────────────────────────────────────────────────────────────────────
📊 Fold 1/5
──────────────────────────────────────────────────────────────────────

[데이터 분할]
  학습: 6,679개 (Low Conf: 1227개)
  검증: 1,670개



In [10]:
# ============================================================
# 단계 3: 특징 중요도 분석 (최종 개선 버전)
# ============================================================

import shap
import matplotlib.pyplot as plt
import seaborn as sns
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

print("\n" + "=" * 70)
print("단계 3: 특징 중요도 분석 (3-Model Ensemble 기반)")
print("=" * 70)
print("\n[분석 방법]")
print("  1. ✓ 3개 모델(LGBM, XGB, CAT)의 Feature Importance 통합")
print("  2. ✓ 5-Fold Cross-Validation 기반")
print("  3. ✓ SHAP 값 분석 (상위 피처)")
print("  4. ✓ 안정성 분석 (표준편차)")

# ============================================================
# 3.1 데이터 준비
# ============================================================
print(f"\n{'='*70}")
print("데이터 준비")
print(f"{'='*70}")

# 원본 데이터 로드 (df가 없으면)
try:
    df = pd.read_csv('train.csv')
except:
    print("⚠️  train.csv 파일이 필요합니다.")

# 컬럼 그룹 정의
fp_cols = [col for col in df.columns if col.startswith(('ecfp_', 'fcfp_', 'ptfp_'))]
desc_cols = ['MolWt', 'clogp', 'sa_score', 'qed']
label_col = 'label'

# Feature names (전처리 후 순서)
feature_names = fp_cols + desc_cols

print(f"\n[피처 구성]")
print(f"  전체 피처: {len(feature_names)}개")
print(f"  - Fingerprint: {len(fp_cols)}개")
print(f"    · ECFP: {len([c for c in fp_cols if c.startswith('ecfp_')])}개")
print(f"    · FCFP: {len([c for c in fp_cols if c.startswith('fcfp_')])}개")
print(f"    · PTFP: {len([c for c in fp_cols if c.startswith('ptfp_')])}개")
print(f"  - Descriptor: {len(desc_cols)}개")

# X, y 분리
X = df.drop(columns=[label_col])
y = df[label_col].astype(int)

# 전처리 파이프라인
preprocessor = ColumnTransformer(
    transformers=[
        ('fp', SimpleImputer(strategy='constant', fill_value=0), fp_cols),
        ('desc', Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), desc_cols)
    ],
    remainder='drop'
)

# 교차검증 설정
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# ============================================================
# 3.2 3-Model Ensemble Feature Importance 계산
# ============================================================
print(f"\n{'='*70}")
print("3-Model Ensemble Feature Importance 계산")
print(f"{'='*70}")

# 결과 저장용
importance_results = {
    'lgbm': {'fold_importances': [], 'models': []},
    'xgb': {'fold_importances': [], 'models': []},
    'catboost': {'fold_importances': [], 'models': []},
}

for fold, (tr_idx, va_idx) in enumerate(skf.split(X, y), 1):
    print(f"\rFold {fold}/5 처리 중...", end='')

    X_tr = X.iloc[tr_idx]
    y_tr = y.iloc[tr_idx]

    # 전처리
    Xt_tr = preprocessor.fit_transform(X_tr)

    # ========================================
    # LightGBM
    # ========================================
    lgbm_model = LGBMClassifier(
        n_estimators=1000, learning_rate=0.03, max_depth=8,
        num_leaves=63, min_child_samples=30, subsample=0.8,
        colsample_bytree=0.8, reg_alpha=0.3, reg_lambda=0.3,
        class_weight={0: 1.5, 1: 1.0},
        random_state=42, n_jobs=-1, verbose=-1
    )
    lgbm_model.fit(Xt_tr, y_tr)

    # Gain 기반 중요도
    lgbm_importances = lgbm_model.booster_.feature_importance(importance_type='gain')
    importance_results['lgbm']['fold_importances'].append(lgbm_importances)
    importance_results['lgbm']['models'].append(lgbm_model)

    # ========================================
    # XGBoost
    # ========================================
    xgb_model = XGBClassifier(
        n_estimators=1000, learning_rate=0.03, max_depth=7,
        min_child_weight=3, subsample=0.8, colsample_bytree=0.8,
        gamma=0.1, reg_alpha=0.3, reg_lambda=0.3,
        scale_pos_weight=0.67,
        random_state=42, n_jobs=-1, verbosity=0
    )
    xgb_model.fit(Xt_tr, y_tr)

    # Gain 기반 중요도
    xgb_importances = xgb_model.feature_importances_
    importance_results['xgb']['fold_importances'].append(xgb_importances)
    importance_results['xgb']['models'].append(xgb_model)

    # ========================================
    # CatBoost
    # ========================================
    cat_model = CatBoostClassifier(
        iterations=1000, learning_rate=0.03, depth=7,
        l2_leaf_reg=3, class_weights=[1.5, 1.0],
        random_seed=42, verbose=0
    )
    cat_model.fit(Xt_tr, y_tr)

    # Feature importance
    cat_importances = cat_model.get_feature_importance()
    importance_results['catboost']['fold_importances'].append(cat_importances)
    importance_results['catboost']['models'].append(cat_model)

print(f"\r✓ 5-Fold Feature Importance 계산 완료")

# ============================================================
# 3.3 모델별 중요도 통계 및 Ensemble
# ============================================================
print(f"\n{'='*70}")
print("모델별 Feature Importance 통합")
print(f"{'='*70}")

# 각 모델별 평균 및 표준편차
lgbm_mean = np.mean(importance_results['lgbm']['fold_importances'], axis=0)
lgbm_std = np.std(importance_results['lgbm']['fold_importances'], axis=0)

xgb_mean = np.mean(importance_results['xgb']['fold_importances'], axis=0)
xgb_std = np.std(importance_results['xgb']['fold_importances'], axis=0)

cat_mean = np.mean(importance_results['catboost']['fold_importances'], axis=0)
cat_std = np.std(importance_results['catboost']['fold_importances'], axis=0)

# Ensemble Importance (XGBoost 중심: 50%)
ensemble_mean = 0.25 * lgbm_mean + 0.50 * xgb_mean + 0.25 * cat_mean
ensemble_std = np.sqrt(
    (0.25 * lgbm_std)**2 +
    (0.50 * xgb_std)**2 +
    (0.25 * cat_std)**2
)

# DataFrame 생성
importance_df = pd.DataFrame({
    'feature': feature_names,
    'ensemble_mean': ensemble_mean,
    'ensemble_std': ensemble_std,
    'lgbm_mean': lgbm_mean,
    'lgbm_std': lgbm_std,
    'xgb_mean': xgb_mean,
    'xgb_std': xgb_std,
    'cat_mean': cat_mean,
    'cat_std': cat_std,
    'cv_coefficient': ensemble_std / (ensemble_mean + 1e-10)  # 변동계수
}).sort_values('ensemble_mean', ascending=False)

print(f"\n[상위 20개 중요 피처 (Ensemble 기준)]")
print(importance_df.head(20)[['feature', 'ensemble_mean', 'ensemble_std', 'cv_coefficient']].to_string(index=False))

# ============================================================
# 3.4 물성 Descriptor 분석
# ============================================================
print(f"\n{'='*70}")
print("물성 Descriptor 중요도 분석")
print(f"{'='*70}")

desc_importance = importance_df[importance_df['feature'].isin(desc_cols)].copy()
desc_importance = desc_importance.sort_values('ensemble_mean', ascending=False)

print(f"\n[Descriptor 중요도 순위]")
for idx, row in desc_importance.iterrows():
    rank = importance_df.index.get_loc(idx) + 1
    print(f"  {rank:3d}위. {row['feature']:<12} : {row['ensemble_mean']:>10.2f} ± {row['ensemble_std']:>6.2f}")

# Descriptor 간 상대 비율
total_desc_importance = desc_importance['ensemble_mean'].sum()
print(f"\n[Descriptor 상대 기여도]")
for idx, row in desc_importance.iterrows():
    ratio = row['ensemble_mean'] / total_desc_importance * 100
    print(f"  {row['feature']:<12} : {ratio:5.1f}%")

# ============================================================
# 3.5 Fingerprint 타입별 분석
# ============================================================
print(f"\n{'='*70}")
print("Fingerprint 타입별 중요도 분석")
print(f"{'='*70}")

fp_importance = importance_df[importance_df['feature'].isin(fp_cols)].copy()

# 타입별 통계
fp_types = {
    'ECFP': [f for f in fp_cols if f.startswith('ecfp_')],
    'FCFP': [f for f in fp_cols if f.startswith('fcfp_')],
    'PTFP': [f for f in fp_cols if f.startswith('ptfp_')]
}

print(f"\n[Fingerprint 타입별 통계]")
print(f"{'타입':<8} {'평균 중요도':<15} {'Top 10 개수':<12} {'Top 50 개수':<12}")
print(f"{'-'*50}")

top10_features = importance_df.head(10)['feature'].tolist()
top50_features = importance_df.head(50)['feature'].tolist()

for fp_type, fp_list in fp_types.items():
    type_importance = importance_df[importance_df['feature'].isin(fp_list)]
    avg_importance = type_importance['ensemble_mean'].mean()
    count_top10 = sum(1 for f in fp_list if f in top10_features)
    count_top50 = sum(1 for f in fp_list if f in top50_features)

    print(f"{fp_type:<8} {avg_importance:<15.2f} {count_top10:<12} {count_top50:<12}")

# ============================================================
# 3.6 Feature Selection 전략
# ============================================================
print(f"\n{'='*70}")
print("Feature Selection 전략")
print(f"{'='*70}")

# 전략 1: 상위 N개 선택
top_n_options = [50, 100, 150, 200]
print(f"\n[전략 1: 상위 N개 선택]")
print(f"{'N':<8} {'누적 중요도':<15} {'압축률':<10}")
print(f"{'-'*35}")

total_importance = importance_df['ensemble_mean'].sum()
for n in top_n_options:
    cumsum = importance_df.head(n)['ensemble_mean'].sum()
    ratio = cumsum / total_importance * 100
    compression = n / len(feature_names) * 100
    print(f"{n:<8} {ratio:<15.2f}% {compression:<10.1f}%")

# 전략 2: 임계값 기반 선택
threshold_options = [10, 20, 30, 50, 100]
print(f"\n[전략 2: 임계값 기반 선택]")
print(f"{'임계값':<10} {'선택 피처 수':<15} {'압축률':<10}")
print(f"{'-'*35}")

for thresh in threshold_options:
    selected = importance_df[importance_df['ensemble_mean'] >= thresh]
    count = len(selected)
    compression = count / len(feature_names) * 100
    print(f"{thresh:<10} {count:<15} {compression:<10.1f}%")

# 전략 3: 누적 기여도 기반
cumsum_options = [80, 90, 95, 99]
print(f"\n[전략 3: 누적 기여도 기반]")
print(f"{'누적 %':<10} {'필요 피처 수':<15} {'압축률':<10}")
print(f"{'-'*35}")

importance_df['cumsum'] = importance_df['ensemble_mean'].cumsum()
importance_df['cumsum_pct'] = importance_df['cumsum'] / total_importance * 100

for target_pct in cumsum_options:
    needed = len(importance_df[importance_df['cumsum_pct'] <= target_pct]) + 1
    compression = needed / len(feature_names) * 100
    print(f"{target_pct:<10} {needed:<15} {compression:<10.1f}%")

# 권장 설정
recommended_n = 150  # 상위 150개
recommended_features = importance_df.head(recommended_n)['feature'].tolist()

print(f"\n[권장 설정]")
print(f"  선택 피처: 상위 {recommended_n}개")
print(f"  누적 중요도: {importance_df.head(recommended_n)['ensemble_mean'].sum() / total_importance * 100:.1f}%")
print(f"  압축률: {recommended_n / len(feature_names) * 100:.1f}%")
print(f"  - Fingerprint: {len([f for f in recommended_features if f in fp_cols])}개")
print(f"  - Descriptor: {len([f for f in recommended_features if f in desc_cols])}개")

# ============================================================
# 3.7 안정성 분석
# ============================================================
print(f"\n{'='*70}")
print("Feature Importance 안정성 분석")
print(f"{'='*70}")

# 변동계수(CV) 분석
high_cv = importance_df[importance_df['cv_coefficient'] > 0.5].sort_values('cv_coefficient', ascending=False)

print(f"\n[높은 변동성 피처 (CV > 0.5)]")
print(f"  불안정한 피처: {len(high_cv)}개")
if len(high_cv) > 0:
    print(f"\n  상위 10개:")
    print(high_cv.head(10)[['feature', 'ensemble_mean', 'ensemble_std', 'cv_coefficient']].to_string(index=False))

# 안정적인 상위 피처
stable_important = importance_df[
    (importance_df['ensemble_mean'] >= 50) &
    (importance_df['cv_coefficient'] <= 0.3)
]

print(f"\n[안정적이면서 중요한 피처 (Importance ≥ 50, CV ≤ 0.3)]")
print(f"  개수: {len(stable_important)}개")
if len(stable_important) > 0:
    print(f"\n  Top 10:")
    print(stable_important.head(10)[['feature', 'ensemble_mean', 'cv_coefficient']].to_string(index=False))

# ============================================================
# 3.8 SHAP 분석 (상위 피처)
# ============================================================
print(f"\n{'='*70}")
print("SHAP 값 분석 (상위 20개 피처)")
print(f"{'='*70}")

# 첫 번째 Fold의 첫 번째 모델(LightGBM) 사용
sample_model = importance_results['lgbm']['models'][0]
X_sample = X.iloc[:1000]  # 샘플 1000개만 (속도)
Xt_sample = preprocessor.fit_transform(X_sample)

# 상위 20개 피처만 선택
top20_indices = importance_df.head(20).index.tolist()
top20_features = importance_df.head(20)['feature'].tolist()

print(f"\n  샘플 수: {len(X_sample)}개")
print(f"  분석 피처: 상위 20개")

try:
    # SHAP Explainer
    explainer = shap.TreeExplainer(sample_model)
    shap_values = explainer.shap_values(Xt_sample)

    # Class 1 (무독성)에 대한 SHAP 값
    if isinstance(shap_values, list):
        shap_values_class1 = shap_values[1]
    else:
        shap_values_class1 = shap_values

    # 상위 20개만 추출
    shap_values_top20 = shap_values_class1[:, top20_indices]

    # SHAP 평균 절대값
    shap_mean_abs = np.abs(shap_values_top20).mean(axis=0)

    print(f"\n[SHAP 평균 절대값 (상위 10개)]")
    shap_df = pd.DataFrame({
        'feature': top20_features,
        'shap_mean_abs': shap_mean_abs
    }).sort_values('shap_mean_abs', ascending=False)

    print(shap_df.head(10).to_string(index=False))

    shap_available = True

except Exception as e:
    print(f"\n⚠️  SHAP 분석 오류: {e}")
    print("  → Feature Importance 분석은 정상 완료됨")
    shap_available = False

# ============================================================
# 3.9 시각화
# ============================================================
print(f"\n{'='*70}")
print("시각화 생성")
print(f"{'='*70}")

fig, axes = plt.subplots(2, 3, figsize=(20, 12))

# 1. Top 20 Feature Importance (Ensemble)
ax = axes[0, 0]
top20 = importance_df.head(20)
colors = ['#1f77b4' if f in desc_cols else '#ff7f0e' for f in top20['feature']]
ax.barh(range(len(top20)), top20['ensemble_mean'], color=colors, alpha=0.7, edgecolor='black')
ax.set_yticks(range(len(top20)))
ax.set_yticklabels(top20['feature'], fontsize=9)
ax.invert_yaxis()
ax.set_xlabel('Ensemble Importance (Gain)', fontsize=11)
ax.set_title('Top 20 Feature Importance (Ensemble)', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3, axis='x')

# 범례
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='#1f77b4', label='Descriptor'),
    Patch(facecolor='#ff7f0e', label='Fingerprint')
]
ax.legend(handles=legend_elements, loc='lower right')

# 2. 모델별 비교 (상위 10개)
ax = axes[0, 1]
top10 = importance_df.head(10)
x = np.arange(len(top10))
width = 0.25

ax.barh(x - width, top10['lgbm_mean'], width, label='LightGBM', alpha=0.8)
ax.barh(x, top10['xgb_mean'], width, label='XGBoost', alpha=0.8)
ax.barh(x + width, top10['cat_mean'], width, label='CatBoost', alpha=0.8)

ax.set_yticks(x)
ax.set_yticklabels(top10['feature'], fontsize=9)
ax.invert_yaxis()
ax.set_xlabel('Importance (Gain)', fontsize=11)
ax.set_title('Top 10 Features by Model', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3, axis='x')

# 3. 안정성 분석 (상위 30개)
ax = axes[0, 2]
top30 = importance_df.head(30)
scatter = ax.scatter(
    top30['ensemble_mean'],
    top30['cv_coefficient'],
    s=100, alpha=0.6, c=range(len(top30)), cmap='viridis',
    edgecolors='black', linewidth=1
)
ax.axhline(y=0.3, color='r', linestyle='--', label='Stability Threshold (0.3)')
ax.set_xlabel('Ensemble Importance', fontsize=11)
ax.set_ylabel('Coefficient of Variation (CV)', fontsize=11)
ax.set_title('Importance vs Stability (Top 30)', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# 상위 5개 라벨링
for idx in range(min(5, len(top30))):
    row = top30.iloc[idx]
    ax.annotate(
        row['feature'],
        (row['ensemble_mean'], row['cv_coefficient']),
        fontsize=8, alpha=0.7,
        xytext=(5, 5), textcoords='offset points'
    )

# 4. 누적 중요도
ax = axes[1, 0]
cumsum_pct = importance_df['cumsum_pct'].values
ax.plot(range(1, len(cumsum_pct)+1), cumsum_pct, linewidth=2)
ax.axhline(y=90, color='r', linestyle='--', label='90%')
ax.axhline(y=95, color='orange', linestyle='--', label='95%')
ax.axvline(x=recommended_n, color='g', linestyle='--', label=f'Top {recommended_n}')
ax.set_xlabel('Number of Features', fontsize=11)
ax.set_ylabel('Cumulative Importance (%)', fontsize=11)
ax.set_title('Cumulative Feature Importance', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_xlim([0, 500])

# 5. Fingerprint 타입별 분포
ax = axes[1, 1]
fp_type_data = []
fp_type_labels = []
for fp_type, fp_list in fp_types.items():
    type_importance = importance_df[importance_df['feature'].isin(fp_list)]['ensemble_mean'].values
    fp_type_data.append(type_importance)
    fp_type_labels.append(fp_type)

bp = ax.boxplot(fp_type_data, labels=fp_type_labels, patch_artist=True)
for patch, color in zip(bp['boxes'], ['lightblue', 'lightgreen', 'lightcoral']):
    patch.set_facecolor(color)
ax.set_ylabel('Ensemble Importance', fontsize=11)
ax.set_title('Fingerprint Type Distribution', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')

# 6. Descriptor 중요도
ax = axes[1, 2]
desc_data = desc_importance[['feature', 'ensemble_mean', 'ensemble_std']].copy()
x_pos = np.arange(len(desc_data))
ax.bar(x_pos, desc_data['ensemble_mean'], yerr=desc_data['ensemble_std'],
       alpha=0.7, capsize=5, edgecolor='black', color='steelblue')
ax.set_xticks(x_pos)
ax.set_xticklabels(desc_data['feature'], fontsize=10, rotation=0)
ax.set_ylabel('Ensemble Importance (Gain)', fontsize=11)
ax.set_title('Descriptor Importance', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('feature_importance_comprehensive.png', dpi=300, bbox_inches='tight')
plt.close()
print(f"\n✓ 종합 시각화 저장: feature_importance_comprehensive.png")

# 상위 20개만 별도 저장
fig, ax = plt.subplots(figsize=(10, 8))
top20 = importance_df.head(20)
colors = ['#2E86AB' if f in desc_cols else '#A23B72' for f in top20['feature']]
bars = ax.barh(range(len(top20)), top20['ensemble_mean'],
               xerr=top20['ensemble_std'], color=colors, alpha=0.8,
               edgecolor='black', linewidth=1.5, capsize=3)
ax.set_yticks(range(len(top20)))
ax.set_yticklabels(top20['feature'], fontsize=11, fontweight='bold')
ax.invert_yaxis()
ax.set_xlabel('Ensemble Importance (Gain)', fontsize=12, fontweight='bold')
ax.set_title('Top 20 Feature Importance\n(3-Model Ensemble, 5-Fold CV)',
             fontsize=14, fontweight='bold', pad=20)
ax.grid(True, alpha=0.3, axis='x')

# 범례
legend_elements = [
    Patch(facecolor='#2E86AB', label='Descriptor (물성)'),
    Patch(facecolor='#A23B72', label='Fingerprint (구조)')
]
ax.legend(handles=legend_elements, loc='lower right', fontsize=10)

plt.tight_layout()
plt.savefig('feature_importance_top20.png', dpi=300, bbox_inches='tight')
plt.close()
print(f"✓ Top 20 시각화 저장: feature_importance_top20.png")

# ============================================================
# 3.10 결과 저장
# ============================================================
print(f"\n{'='*70}")
print("결과 저장")
print(f"{'='*70}")

# 전체 결과 저장
importance_df.to_csv('feature_importance_ensemble_cv.csv', index=False)
print(f"✓ 전체 Feature Importance 저장: feature_importance_ensemble_cv.csv")

# 권장 피처 저장
recommended_df = importance_df.head(recommended_n)[['feature', 'ensemble_mean', 'ensemble_std']]
recommended_df.to_csv('selected_features_top150.csv', index=False)
print(f"✓ 권장 피처 저장 (Top {recommended_n}): selected_features_top150.csv")

# 안정적 피처 저장
if len(stable_important) > 0:
    stable_important[['feature', 'ensemble_mean', 'cv_coefficient']].to_csv(
        'stable_important_features.csv', index=False
    )
    print(f"✓ 안정적 중요 피처 저장: stable_important_features.csv")

print(f"\n{'='*70}")
print("✓ 단계 3 완료 - Feature Importance 분석 완료")
print(f"{'='*70}")

# 전역 변수 저장
globals().update({
    'feature_importance_df': importance_df,
    'recommended_features': recommended_features,
    'stable_important_features': stable_important['feature'].tolist() if len(stable_important) > 0 else []
})



단계 3: 특징 중요도 분석 (3-Model Ensemble 기반)

[분석 방법]
  1. ✓ 3개 모델(LGBM, XGB, CAT)의 Feature Importance 통합
  2. ✓ 5-Fold Cross-Validation 기반
  3. ✓ SHAP 값 분석 (상위 피처)
  4. ✓ 안정성 분석 (표준편차)

데이터 준비

[피처 구성]
  전체 피처: 3076개
  - Fingerprint: 3072개
    · ECFP: 1024개
    · FCFP: 1024개
    · PTFP: 1024개
  - Descriptor: 4개

3-Model Ensemble Feature Importance 계산
✓ 5-Fold Feature Importance 계산 완료

모델별 Feature Importance 통합

[상위 20개 중요 피처 (Ensemble 기준)]
  feature  ensemble_mean  ensemble_std  cv_coefficient
    clogp    4553.962173    226.119807        0.049653
 ecfp_807    1051.635186    120.216099        0.114313
      qed     969.271712     51.724258        0.053364
 sa_score     930.372308     41.783955        0.044911
    MolWt     832.295959     56.295232        0.067638
  fcfp_18     591.584150    100.075438        0.169165
 fcfp_926     568.021539     93.016768        0.163756
 ecfp_893     270.677769     87.739696        0.324148
 ecfp_219     254.949614     76.201583        0.298889
 ecfp_887  

In [11]:
# ============================================================
# 단계 4: Top 150 피처 기반 최종 모델 학습 및 Threshold 최적화
# ============================================================

import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    f1_score, precision_recall_curve, roc_curve,
    roc_auc_score, average_precision_score,
    classification_report, confusion_matrix,
    precision_score, recall_score
)
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

print("=" * 70)
print("단계 4: Top 150 피처 기반 최종 모델 학습")
print("=" * 70)
print("\n[목표]")
print("  1. ✓ Top 150 피처로 3-Model Ensemble 재학습")
print("  2. ✓ Threshold 재최적화 (FPR ≤ 25%)")
print("  3. ✓ 성능 향상 검증")
print("  4. ✓ Test 데이터 예측 준비")

# ============================================================
# 4.1 Top 150 피처 로드 및 데이터 준비
# ============================================================
print(f"\n{'='*70}")
print("데이터 준비")
print(f"{'='*70}")

# Top 150 피처 로드
selected_features_df = pd.read_csv('selected_features_top150.csv')
selected_features = selected_features_df['feature'].tolist()

print(f"\n[피처 선택]")
print(f"  선택된 피처: {len(selected_features)}개")
print(f"  - Fingerprint: {len([f for f in selected_features if f.startswith(('ecfp_', 'fcfp_', 'ptfp_'))])}개")
print(f"  - Descriptor: {len([f for f in selected_features if f in ['MolWt', 'clogp', 'sa_score', 'qed']])}개")

# 데이터 로드
df = pd.read_csv('train.csv')
X = df[selected_features]
y = df['label'].astype(int)

print(f"\n[데이터 크기]")
print(f"  이전: (8349, 3076)")
print(f"  현재: {X.shape}")
print(f"  압축률: {X.shape[1]/3076*100:.1f}% ({3076-X.shape[1]}개 제거)")

# ============================================================
# 4.2 전처리 파이프라인
# ============================================================
fp_cols = [f for f in selected_features if f.startswith(('ecfp_', 'fcfp_', 'ptfp_'))]
desc_cols = ['MolWt', 'clogp', 'sa_score', 'qed']

preprocessor = ColumnTransformer(
    transformers=[
        ('fp', SimpleImputer(strategy='constant', fill_value=0), fp_cols),
        ('desc', Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), desc_cols)
    ],
    remainder='drop'
)

# 교차검증
RANDOM_STATE = 42
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

# ============================================================
# 4.3 1단계: OOF 확률 수집 (Threshold 최적화용)
# ============================================================
print(f"\n{'='*70}")
print("1단계: OOF 확률 수집")
print(f"{'='*70}")

results_stage1 = {
    'lgbm': {'oof_probabilities': np.zeros(len(X))},
    'xgb': {'oof_probabilities': np.zeros(len(X))},
    'catboost': {'oof_probabilities': np.zeros(len(X))},
    'ensemble': {'oof_probabilities': np.zeros(len(X))}
}

for fold, (tr_idx, va_idx) in enumerate(skf.split(X, y), 1):
    print(f"\rFold {fold}/5 처리 중...", end='')

    X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
    y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]

    Xt_tr = preprocessor.fit_transform(X_tr)
    Xt_va = preprocessor.transform(X_va)

    # LightGBM
    lgbm_model = LGBMClassifier(
        n_estimators=1000, learning_rate=0.03, max_depth=8,
        num_leaves=63, min_child_samples=30, subsample=0.8,
        colsample_bytree=0.8, reg_alpha=0.3, reg_lambda=0.3,
        class_weight={0: 1.5, 1: 1.0},
        random_state=RANDOM_STATE, n_jobs=-1, verbose=-1
    )
    lgbm_model.fit(Xt_tr, y_tr, eval_set=[(Xt_va, y_va)],
                   callbacks=[lgb.early_stopping(stopping_rounds=100, verbose=False)])
    results_stage1['lgbm']['oof_probabilities'][va_idx] = lgbm_model.predict_proba(Xt_va)[:, 1]

    # XGBoost
    xgb_model = XGBClassifier(
        n_estimators=1000, learning_rate=0.03, max_depth=7,
        min_child_weight=3, subsample=0.8, colsample_bytree=0.8,
        gamma=0.1, reg_alpha=0.3, reg_lambda=0.3,
        scale_pos_weight=0.67,
        random_state=RANDOM_STATE, n_jobs=-1,
        early_stopping_rounds=100, eval_metric='logloss', verbosity=0
    )
    xgb_model.fit(Xt_tr, y_tr, eval_set=[(Xt_va, y_va)], verbose=False)
    results_stage1['xgb']['oof_probabilities'][va_idx] = xgb_model.predict_proba(Xt_va)[:, 1]

    # CatBoost
    cat_model = CatBoostClassifier(
        iterations=1000, learning_rate=0.03, depth=7,
        l2_leaf_reg=3, class_weights=[1.5, 1.0],
        random_seed=RANDOM_STATE, verbose=0,
        early_stopping_rounds=100
    )
    cat_model.fit(Xt_tr, y_tr, eval_set=(Xt_va, y_va), verbose=False)
    results_stage1['catboost']['oof_probabilities'][va_idx] = cat_model.predict_proba(Xt_va)[:, 1]

print(f"\r✓ 1단계 완료: OOF 확률 수집 완료")

# Ensemble
results_stage1['ensemble']['oof_probabilities'] = (
    0.25 * results_stage1['lgbm']['oof_probabilities'] +
    0.50 * results_stage1['xgb']['oof_probabilities'] +
    0.25 * results_stage1['catboost']['oof_probabilities']
)

# ============================================================
# 4.4 Threshold 재최적화
# ============================================================
print(f"\n{'='*70}")
print("2단계: Threshold 재최적화")
print(f"{'='*70}")

def optimize_threshold_with_fpr(y_true, y_pred_proba, max_fpr=0.25):
    """FPR 제약 하에서 F1 Score 최적화"""
    thresholds = np.arange(0.1, 0.9, 0.005)
    best_f1 = 0
    best_threshold = 0.5
    best_fpr = 1.0

    results_list = []

    for thresh in thresholds:
        y_pred = (y_pred_proba >= thresh).astype(int)
        f1 = f1_score(y_true, y_pred)

        cm = confusion_matrix(y_true, y_pred)
        tn, fp, fn, tp = cm.ravel()
        fpr = fp / (fp + tn) if (fp + tn) > 0 else 0

        results_list.append({
            'threshold': thresh,
            'f1': f1,
            'fpr': fpr,
            'precision': tp / (tp + fp) if (tp + fp) > 0 else 0,
            'recall': tp / (tp + fn) if (tp + fn) > 0 else 0
        })

        if fpr <= max_fpr and f1 > best_f1:
            best_f1 = f1
            best_threshold = thresh
            best_fpr = fpr

    return best_threshold, best_f1, best_fpr, results_list

# Threshold 탐색
optimal_threshold, optimal_f1, optimal_fpr, threshold_results = optimize_threshold_with_fpr(
    y,
    results_stage1['ensemble']['oof_probabilities'],
    max_fpr=0.25
)

print(f"\n[최적 Threshold 탐색 결과]")
print(f"  제약 조건: FPR ≤ 25%")
print(f"  최적 Threshold: {optimal_threshold:.3f}")
print(f"  예상 F1 Score: {optimal_f1:.4f}")
print(f"  예상 FPR: {optimal_fpr:.4f} ({optimal_fpr*100:.2f}%)")

# 이전과 비교
print(f"\n[이전 대비 비교]")
print(f"  이전 Threshold: 0.385 (2단계)")
print(f"  현재 Threshold: {optimal_threshold:.3f} (Top 150 기반)")
print(f"  변화: {optimal_threshold - 0.385:+.3f}")

# ============================================================
# 4.5 최종 모델 학습 및 평가
# ============================================================
print(f"\n{'='*70}")
print("3단계: 최종 3-Model Ensemble 학습")
print(f"{'='*70}")

results_final = {
    'lgbm': {'f1_scores': [], 'auc_scores': [], 'models': [],
             'oof_predictions': np.zeros(len(X)),
             'oof_probabilities': np.zeros(len(X))},
    'xgb': {'f1_scores': [], 'auc_scores': [], 'models': [],
            'oof_predictions': np.zeros(len(X)),
            'oof_probabilities': np.zeros(len(X))},
    'catboost': {'f1_scores': [], 'auc_scores': [], 'models': [],
                 'oof_predictions': np.zeros(len(X)),
                 'oof_probabilities': np.zeros(len(X))},
    'ensemble': {'oof_probabilities': np.zeros(len(X)),
                 'oof_predictions': np.zeros(len(X))},
    'fold_details': []
}

for fold, (tr_idx, va_idx) in enumerate(skf.split(X, y), 1):
    print(f"\n{'─'*70}")
    print(f"📊 Fold {fold}/5")
    print(f"{'─'*70}")

    X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
    y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]

    Xt_tr = preprocessor.fit_transform(X_tr)
    Xt_va = preprocessor.transform(X_va)

    print(f"  학습: {Xt_tr.shape}, 검증: {Xt_va.shape}")

    # LightGBM
    print(f"  [1/3] LightGBM...", end=' ')
    lgbm_model = LGBMClassifier(
        n_estimators=1000, learning_rate=0.03, max_depth=8,
        num_leaves=63, min_child_samples=30, subsample=0.8,
        colsample_bytree=0.8, reg_alpha=0.3, reg_lambda=0.3,
        class_weight={0: 1.5, 1: 1.0},
        random_state=RANDOM_STATE, n_jobs=-1, verbose=-1
    )
    lgbm_model.fit(Xt_tr, y_tr, eval_set=[(Xt_va, y_va)],
                   callbacks=[lgb.early_stopping(stopping_rounds=100, verbose=False)])

    lgbm_proba = lgbm_model.predict_proba(Xt_va)[:, 1]
    lgbm_pred = (lgbm_proba >= optimal_threshold).astype(int)
    lgbm_f1 = f1_score(y_va, lgbm_pred)

    results_final['lgbm']['models'].append(lgbm_model)
    results_final['lgbm']['oof_probabilities'][va_idx] = lgbm_proba
    results_final['lgbm']['oof_predictions'][va_idx] = lgbm_pred
    results_final['lgbm']['f1_scores'].append(lgbm_f1)
    results_final['lgbm']['auc_scores'].append(roc_auc_score(y_va, lgbm_proba))

    print(f"F1: {lgbm_f1:.4f}")

    # XGBoost
    print(f"  [2/3] XGBoost...", end=' ')
    xgb_model = XGBClassifier(
        n_estimators=1000, learning_rate=0.03, max_depth=7,
        min_child_weight=3, subsample=0.8, colsample_bytree=0.8,
        gamma=0.1, reg_alpha=0.3, reg_lambda=0.3,
        scale_pos_weight=0.67,
        random_state=RANDOM_STATE, n_jobs=-1,
        early_stopping_rounds=100, eval_metric='logloss', verbosity=0
    )
    xgb_model.fit(Xt_tr, y_tr, eval_set=[(Xt_va, y_va)], verbose=False)

    xgb_proba = xgb_model.predict_proba(Xt_va)[:, 1]
    xgb_pred = (xgb_proba >= optimal_threshold).astype(int)
    xgb_f1 = f1_score(y_va, xgb_pred)

    results_final['xgb']['models'].append(xgb_model)
    results_final['xgb']['oof_probabilities'][va_idx] = xgb_proba
    results_final['xgb']['oof_predictions'][va_idx] = xgb_pred
    results_final['xgb']['f1_scores'].append(xgb_f1)
    results_final['xgb']['auc_scores'].append(roc_auc_score(y_va, xgb_proba))

    print(f"F1: {xgb_f1:.4f}")

    # CatBoost
    print(f"  [3/3] CatBoost...", end=' ')
    cat_model = CatBoostClassifier(
        iterations=1000, learning_rate=0.03, depth=7,
        l2_leaf_reg=3, class_weights=[1.5, 1.0],
        random_seed=RANDOM_STATE, verbose=0,
        early_stopping_rounds=100
    )
    cat_model.fit(Xt_tr, y_tr, eval_set=(Xt_va, y_va), verbose=False)

    cat_proba = cat_model.predict_proba(Xt_va)[:, 1]
    cat_pred = (cat_proba >= optimal_threshold).astype(int)
    cat_f1 = f1_score(y_va, cat_pred)

    results_final['catboost']['models'].append(cat_model)
    results_final['catboost']['oof_probabilities'][va_idx] = cat_proba
    results_final['catboost']['oof_predictions'][va_idx] = cat_pred
    results_final['catboost']['f1_scores'].append(cat_f1)
    results_final['catboost']['auc_scores'].append(roc_auc_score(y_va, cat_proba))

    print(f"F1: {cat_f1:.4f}")

    # Ensemble
    ensemble_proba = 0.25 * lgbm_proba + 0.50 * xgb_proba + 0.25 * cat_proba
    ensemble_pred = (ensemble_proba >= optimal_threshold).astype(int)

    results_final['ensemble']['oof_probabilities'][va_idx] = ensemble_proba
    results_final['ensemble']['oof_predictions'][va_idx] = ensemble_pred

    ensemble_f1 = f1_score(y_va, ensemble_pred)
    ensemble_auc = roc_auc_score(y_va, ensemble_proba)

    cm = confusion_matrix(y_va, ensemble_pred)
    tn, fp, fn, tp = cm.ravel()
    fpr = fp / (fp + tn)

    print(f"\n  [Ensemble] F1: {ensemble_f1:.4f}, AUC: {ensemble_auc:.4f}, FPR: {fpr:.4f}")

    results_final['fold_details'].append({
        'fold': fold,
        'lgbm_f1': lgbm_f1, 'xgb_f1': xgb_f1, 'cat_f1': cat_f1,
        'ensemble_f1': ensemble_f1, 'ensemble_auc': ensemble_auc,
        'fpr': fpr
    })

# ============================================================
# 4.6 최종 결과 및 비교
# ============================================================
print(f"\n{'='*70}")
print("최종 결과")
print(f"{'='*70}")

# OOF 성능
ensemble_oof_f1 = f1_score(y, results_final['ensemble']['oof_predictions'])
ensemble_oof_auc = roc_auc_score(y, results_final['ensemble']['oof_probabilities'])

oof_cm = confusion_matrix(y, results_final['ensemble']['oof_predictions'])
tn, fp, fn, tp = oof_cm.ravel()
final_fpr = fp / (fp + tn)
final_precision = tp / (tp + fp)
final_recall = tp / (tp + fn)

print(f"\n[Ensemble OOF 성능]")
print(f"  F1 Score:  {ensemble_oof_f1:.4f}")
print(f"  AUC Score: {ensemble_oof_auc:.4f}")
print(f"  Precision: {final_precision:.4f}")
print(f"  Recall:    {final_recall:.4f}")
print(f"  FPR:       {final_fpr:.4f} ({final_fpr*100:.2f}%)")

print(f"\n[OOF 혼동 행렬]")
print(f"              예측: 0    예측: 1")
print(f"  실제: 0  |   {oof_cm[0,0]:4d}      {oof_cm[0,1]:4d}")
print(f"  실제: 1  |   {oof_cm[1,0]:4d}      {oof_cm[1,1]:4d}")

# 이전 결과와 비교
print(f"\n[성능 비교]")
print(f"{'지표':<15} {'2단계 (3076개)':<18} {'4단계 (150개)':<18} {'변화':<10}")
print(f"{'-'*65}")

baseline_f1 = 0.8317
baseline_fpr = 0.2514

f1_change = ensemble_oof_f1 - baseline_f1
fpr_change = final_fpr - baseline_fpr

print(f"{'F1 Score':<15} {baseline_f1:<18.4f} {ensemble_oof_f1:<18.4f} {f1_change:+.4f}")
print(f"{'FPR':<15} {baseline_fpr:<18.4f} {final_fpr:<18.4f} {fpr_change:+.4f}")
print(f"{'피처 수':<15} {'3076':<18} {'150':<18} {'-2926'}")

# Low Confidence
confidence = np.abs(results_final['ensemble']['oof_probabilities'] - 0.5)
low_conf_mask = confidence < 0.1
n_low_conf = low_conf_mask.sum()
low_conf_acc = (
    results_final['ensemble']['oof_predictions'][low_conf_mask] == y[low_conf_mask]
).mean() if n_low_conf > 0 else 0

print(f"\n[Low Confidence]")
print(f"  이전: 941개 (정확도 57.39%)")
print(f"  현재: {n_low_conf}개 (정확도 {low_conf_acc:.4f})")

# ============================================================
# 4.7 시각화
# ============================================================
print(f"\n{'='*70}")
print("시각화 생성")
print(f"{'='*70}")

fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# 1. F1 Score vs Threshold
ax = axes[0, 0]
thresh_vals = [r['threshold'] for r in threshold_results]
f1_vals = [r['f1'] for r in threshold_results]
ax.plot(thresh_vals, f1_vals, linewidth=2)
ax.axvline(optimal_threshold, color='r', linestyle='--', label=f'Optimal: {optimal_threshold:.3f}')
ax.axvline(0.5, color='gray', linestyle=':', label='Default: 0.5')
ax.set_xlabel('Threshold')
ax.set_ylabel('F1 Score')
ax.set_title('F1 Score vs Threshold (Top 150 Features)')
ax.legend()
ax.grid(True, alpha=0.3)

# 2. ROC Curve
ax = axes[0, 1]
fpr_curve, tpr_curve, _ = roc_curve(y, results_final['ensemble']['oof_probabilities'])
ax.plot(fpr_curve, tpr_curve, linewidth=2, label=f'AUC={ensemble_oof_auc:.4f}')
ax.plot([0, 1], [0, 1], 'k--', alpha=0.3)
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title('ROC Curve (Ensemble OOF)')
ax.legend()
ax.grid(True, alpha=0.3)

# 3. Precision-Recall
ax = axes[0, 2]
precision_curve, recall_curve, _ = precision_recall_curve(
    y, results_final['ensemble']['oof_probabilities']
)
ax.plot(recall_curve, precision_curve, linewidth=2)
ax.set_xlabel('Recall')
ax.set_ylabel('Precision')
ax.set_title('Precision-Recall Curve')
ax.grid(True, alpha=0.3)

# 4. Confidence Distribution
ax = axes[1, 0]
ax.hist(confidence, bins=50, edgecolor='black', alpha=0.7)
ax.axvline(0.1, color='r', linestyle='--', label=f'Low: {n_low_conf}')
ax.set_xlabel('Confidence')
ax.set_ylabel('Frequency')
ax.set_title('Confidence Distribution')
ax.legend()
ax.grid(True, alpha=0.3)

# 5. Confusion Matrix
ax = axes[1, 1]
sns.heatmap(oof_cm, annot=True, fmt='d', cmap='Blues', ax=ax)
ax.set_xlabel('Predicted')
ax.set_ylabel('Actual')
ax.set_title('Confusion Matrix (Ensemble OOF)')

# 6. 성능 비교
ax = axes[1, 2]
models = ['2단계\n(3076개)', '4단계\n(150개)']
f1_values = [baseline_f1, ensemble_oof_f1]
colors = ['lightblue', 'darkgreen']

bars = ax.bar(models, f1_values, color=colors, alpha=0.8, edgecolor='black')
ax.set_ylabel('F1 Score')
ax.set_title('Performance Comparison')
ax.set_ylim([0.82, 0.85])
ax.grid(True, alpha=0.3, axis='y')

for bar, val in zip(bars, f1_values):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{val:.4f}',
            ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.savefig('top150_final_analysis.png', dpi=300, bbox_inches='tight')
plt.close()
print(f"\n✓ 시각화 저장: top150_final_analysis.png")

# ============================================================
# 4.8 결과 저장
# ============================================================
results_df = pd.DataFrame(results_final['fold_details'])
results_df.to_csv('top150_cv_results.csv', index=False)
print(f"✓ CV 결과 저장: top150_cv_results.csv")

print(f"\n{'='*70}")
print("✓ 단계 4 완료 - Top 150 피처 기반 최종 모델 완성")
print(f"{'='*70}")
print(f"\n[최종 권장 설정]")
print(f"  사용 피처: Top 150개")
print(f"  Threshold: {optimal_threshold:.3f}")
print(f"  예상 F1: {ensemble_oof_f1:.4f}")
print(f"  예상 FPR: {final_fpr*100:.2f}%")

# 전역 변수 저장
globals().update({
    'top150_results': results_final,
    'optimal_threshold_final': optimal_threshold,
    'selected_features_final': selected_features
})


단계 4: Top 150 피처 기반 최종 모델 학습

[목표]
  1. ✓ Top 150 피처로 3-Model Ensemble 재학습
  2. ✓ Threshold 재최적화 (FPR ≤ 25%)
  3. ✓ 성능 향상 검증
  4. ✓ Test 데이터 예측 준비

데이터 준비

[피처 선택]
  선택된 피처: 150개
  - Fingerprint: 146개
  - Descriptor: 4개

[데이터 크기]
  이전: (8349, 3076)
  현재: (8349, 150)
  압축률: 4.9% (2926개 제거)

1단계: OOF 확률 수집
✓ 1단계 완료: OOF 확률 수집 완료

2단계: Threshold 재최적화

[최적 Threshold 탐색 결과]
  제약 조건: FPR ≤ 25%
  최적 Threshold: 0.405
  예상 F1 Score: 0.8232
  예상 FPR: 0.2467 (24.67%)

[이전 대비 비교]
  이전 Threshold: 0.385 (2단계)
  현재 Threshold: 0.405 (Top 150 기반)
  변화: +0.020

3단계: 최종 3-Model Ensemble 학습

──────────────────────────────────────────────────────────────────────
📊 Fold 1/5
──────────────────────────────────────────────────────────────────────
  학습: (6679, 150), 검증: (1670, 150)
  [1/3] LightGBM... F1: 0.8311
  [2/3] XGBoost... F1: 0.8379
  [3/3] CatBoost... F1: 0.8356

  [Ensemble] F1: 0.8395, AUC: 0.8997, FPR: 0.2352

──────────────────────────────────────────────────────────────────────
📊 Fold 2/5
───────

In [12]:
# ============================================================
# 단계 5: 최적 피처 수 탐색 (Grid Search)
# ============================================================

import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, roc_auc_score, confusion_matrix
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
import time
import warnings
warnings.filterwarnings('ignore')

print("=" * 70)
print("단계 5: 최적 피처 수 탐색 (Grid Search)")
print("=" * 70)
print("\n[목표]")
print("  1. ✓ 다양한 피처 수 (100~300) 실험")
print("  2. ✓ 성능 vs 효율성 Trade-off 분석")
print("  3. ✓ 최적 피처 수 결정")
print("  4. ✓ 최종 모델 선택")

# ============================================================
# 5.1 데이터 로드
# ============================================================
print(f"\n{'='*70}")
print("데이터 준비")
print(f"{'='*70}")

# Feature importance 로드
importance_df = pd.read_csv('feature_importance_ensemble_cv.csv')
df = pd.read_csv('train.csv')
y = df['label'].astype(int)

# 교차검증 설정
RANDOM_STATE = 42
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

# 테스트할 피처 수 목록
FEATURE_COUNTS = [100, 150, 200, 250, 300]

print(f"\n[실험 설정]")
print(f"  테스트할 피처 수: {FEATURE_COUNTS}")
print(f"  5-Fold Cross-Validation")
print(f"  3-Model Ensemble (LGBM 25%, XGB 50%, CAT 25%)")

# ============================================================
# 5.2 피처 수별 실험 함수
# ============================================================

def train_and_evaluate(n_features, X, y, skf, verbose=True):
    """
    주어진 피처 수로 모델 학습 및 평가
    """
    # 피처 선택
    selected_features = importance_df.head(n_features)['feature'].tolist()
    X_selected = df[selected_features]

    # 전처리 파이프라인
    fp_cols = [f for f in selected_features if f.startswith(('ecfp_', 'fcfp_', 'ptfp_'))]
    desc_cols = [f for f in selected_features if f in ['MolWt', 'clogp', 'sa_score', 'qed']]

    preprocessor = ColumnTransformer(
        transformers=[
            ('fp', SimpleImputer(strategy='constant', fill_value=0), fp_cols),
            ('desc', Pipeline([
                ('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())
            ]), desc_cols)
        ],
        remainder='drop'
    )

    # 결과 저장
    results = {
        'oof_probabilities': np.zeros(len(X_selected)),
        'f1_scores': [],
        'auc_scores': [],
        'fpr_scores': [],
        'train_times': []
    }

    # 5-Fold CV
    for fold, (tr_idx, va_idx) in enumerate(skf.split(X_selected, y), 1):
        if verbose:
            print(f"\r    Fold {fold}/5 처리 중...", end='')

        X_tr, X_va = X_selected.iloc[tr_idx], X_selected.iloc[va_idx]
        y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]

        # 전처리
        Xt_tr = preprocessor.fit_transform(X_tr)
        Xt_va = preprocessor.transform(X_va)

        # 학습 시간 측정 시작
        fold_start_time = time.time()

        # LightGBM
        lgbm_model = LGBMClassifier(
            n_estimators=1000, learning_rate=0.03, max_depth=8,
            num_leaves=63, min_child_samples=30, subsample=0.8,
            colsample_bytree=0.8, reg_alpha=0.3, reg_lambda=0.3,
            class_weight={0: 1.5, 1: 1.0},
            random_state=RANDOM_STATE, n_jobs=-1, verbose=-1
        )
        lgbm_model.fit(Xt_tr, y_tr, eval_set=[(Xt_va, y_va)],
                       callbacks=[lgb.early_stopping(stopping_rounds=100, verbose=False)])
        lgbm_proba = lgbm_model.predict_proba(Xt_va)[:, 1]

        # XGBoost
        xgb_model = XGBClassifier(
            n_estimators=1000, learning_rate=0.03, max_depth=7,
            min_child_weight=3, subsample=0.8, colsample_bytree=0.8,
            gamma=0.1, reg_alpha=0.3, reg_lambda=0.3,
            scale_pos_weight=0.67,
            random_state=RANDOM_STATE, n_jobs=-1,
            early_stopping_rounds=100, eval_metric='logloss', verbosity=0
        )
        xgb_model.fit(Xt_tr, y_tr, eval_set=[(Xt_va, y_va)], verbose=False)
        xgb_proba = xgb_model.predict_proba(Xt_va)[:, 1]

        # CatBoost
        cat_model = CatBoostClassifier(
            iterations=1000, learning_rate=0.03, depth=7,
            l2_leaf_reg=3, class_weights=[1.5, 1.0],
            random_seed=RANDOM_STATE, verbose=0,
            early_stopping_rounds=100
        )
        cat_model.fit(Xt_tr, y_tr, eval_set=(Xt_va, y_va), verbose=False)
        cat_proba = cat_model.predict_proba(Xt_va)[:, 1]

        # Ensemble
        ensemble_proba = 0.25 * lgbm_proba + 0.50 * xgb_proba + 0.25 * cat_proba
        results['oof_probabilities'][va_idx] = ensemble_proba

        # 학습 시간
        fold_time = time.time() - fold_start_time
        results['train_times'].append(fold_time)

    if verbose:
        print(f"\r    ✓ 5-Fold 완료")

    # Threshold 최적화
    def optimize_threshold_with_fpr(y_true, y_pred_proba, max_fpr=0.25):
        thresholds = np.arange(0.1, 0.9, 0.005)
        best_f1 = 0
        best_threshold = 0.5

        for thresh in thresholds:
            y_pred = (y_pred_proba >= thresh).astype(int)
            f1 = f1_score(y_true, y_pred)
            cm = confusion_matrix(y_true, y_pred)
            tn, fp, fn, tp = cm.ravel()
            fpr = fp / (fp + tn) if (fp + tn) > 0 else 0

            if fpr <= max_fpr and f1 > best_f1:
                best_f1 = f1
                best_threshold = thresh

        return best_threshold

    optimal_threshold = optimize_threshold_with_fpr(y, results['oof_probabilities'])

    # 최종 예측
    oof_predictions = (results['oof_probabilities'] >= optimal_threshold).astype(int)

    # 성능 지표
    f1 = f1_score(y, oof_predictions)
    auc = roc_auc_score(y, results['oof_probabilities'])

    cm = confusion_matrix(y, oof_predictions)
    tn, fp, fn, tp = cm.ravel()
    fpr = fp / (fp + tn)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    # Low Confidence
    confidence = np.abs(results['oof_probabilities'] - 0.5)
    low_conf_count = (confidence < 0.1).sum()
    low_conf_acc = (oof_predictions[confidence < 0.1] == y[confidence < 0.1]).mean() if low_conf_count > 0 else 0

    return {
        'n_features': n_features,
        'f1': f1,
        'auc': auc,
        'fpr': fpr,
        'precision': precision,
        'recall': recall,
        'threshold': optimal_threshold,
        'low_conf_count': low_conf_count,
        'low_conf_acc': low_conf_acc,
        'avg_train_time': np.mean(results['train_times']),
        'total_train_time': np.sum(results['train_times']),
        'confusion_matrix': cm
    }

# ============================================================
# 5.3 Grid Search 실행
# ============================================================
print(f"\n{'='*70}")
print("Grid Search 실행")
print(f"{'='*70}")

all_results = []

for n_features in FEATURE_COUNTS:
    print(f"\n[{n_features}개 피처]")

    start_time = time.time()
    result = train_and_evaluate(n_features, df, y, skf, verbose=True)
    total_time = time.time() - start_time

    result['wall_time'] = total_time
    all_results.append(result)

    print(f"  F1 Score:  {result['f1']:.4f}")
    print(f"  AUC:       {result['auc']:.4f}")
    print(f"  FPR:       {result['fpr']:.4f} ({result['fpr']*100:.2f}%)")
    print(f"  Threshold: {result['threshold']:.3f}")
    print(f"  Low Conf:  {result['low_conf_count']}개 (정확도 {result['low_conf_acc']:.4f})")
    print(f"  학습 시간: {total_time:.1f}초")

# ============================================================
# 5.4 결과 분석 및 비교
# ============================================================
print(f"\n{'='*70}")
print("결과 비교")
print(f"{'='*70}")

# DataFrame 생성
results_df = pd.DataFrame(all_results)

print(f"\n[종합 비교표]")
print(f"{'피처 수':<10} {'F1':<10} {'AUC':<10} {'FPR':<10} {'Precision':<12} {'Recall':<10} {'시간(초)':<10}")
print(f"{'-'*80}")
for _, row in results_df.iterrows():
    print(f"{row['n_features']:<10} {row['f1']:<10.4f} {row['auc']:<10.4f} "
          f"{row['fpr']:<10.4f} {row['precision']:<12.4f} {row['recall']:<10.4f} "
          f"{row['wall_time']:<10.1f}")

# 최고 성능 찾기
best_f1_idx = results_df['f1'].idxmax()
best_f1_row = results_df.iloc[best_f1_idx]

print(f"\n[최고 F1 Score]")
print(f"  피처 수: {best_f1_row['n_features']:.0f}개")
print(f"  F1 Score: {best_f1_row['f1']:.4f}")
print(f"  FPR: {best_f1_row['fpr']:.4f} ({best_f1_row['fpr']*100:.2f}%)")

# 최적 Trade-off 찾기 (F1 * 속도)
results_df['efficiency_score'] = results_df['f1'] / (results_df['wall_time'] / results_df['wall_time'].min())
best_tradeoff_idx = results_df['efficiency_score'].idxmax()
best_tradeoff_row = results_df.iloc[best_tradeoff_idx]

print(f"\n[최적 Trade-off (성능 vs 속도)]")
print(f"  피처 수: {best_tradeoff_row['n_features']:.0f}개")
print(f"  F1 Score: {best_tradeoff_row['f1']:.4f}")
print(f"  학습 시간: {best_tradeoff_row['wall_time']:.1f}초")
print(f"  효율성 점수: {best_tradeoff_row['efficiency_score']:.4f}")

# 2단계 (3076개) 기준과 비교
baseline_f1 = 0.8317
baseline_time = 100  # 상대값

print(f"\n[2단계 (3076개 피처) 대비 비교]")
print(f"{'피처 수':<10} {'F1 변화':<15} {'속도 개선':<15} {'종합 평가':<15}")
print(f"{'-'*60}")
for _, row in results_df.iterrows():
    f1_change = row['f1'] - baseline_f1
    f1_change_pct = (f1_change / baseline_f1) * 100
    speed_improvement = (baseline_time - row['wall_time']) / baseline_time * 100

    if f1_change >= 0 and speed_improvement > 80:
        evaluation = "✓✓ 최고"
    elif f1_change >= -0.005 and speed_improvement > 80:
        evaluation = "✓ 우수"
    elif f1_change >= -0.01:
        evaluation = "△ 양호"
    else:
        evaluation = "⚠️ 주의"

    print(f"{row['n_features']:<10} {f1_change:+.4f} ({f1_change_pct:+.2f}%){'':<3} "
          f"{speed_improvement:+.1f}%{'':<8} {evaluation}")

# ============================================================
# 5.5 시각화
# ============================================================
print(f"\n{'='*70}")
print("시각화 생성")
print(f"{'='*70}")

fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# 1. F1 Score vs 피처 수
ax = axes[0, 0]
ax.plot(results_df['n_features'], results_df['f1'], 'o-', linewidth=2, markersize=8)
ax.axhline(baseline_f1, color='r', linestyle='--', label=f'Baseline (3076개): {baseline_f1:.4f}')
ax.set_xlabel('Number of Features', fontsize=11)
ax.set_ylabel('F1 Score', fontsize=11)
ax.set_title('F1 Score vs Feature Count', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# 최고 F1 마킹
ax.scatter([best_f1_row['n_features']], [best_f1_row['f1']],
           color='green', s=200, marker='*', zorder=5, label='Best')

# 2. AUC vs 피처 수
ax = axes[0, 1]
ax.plot(results_df['n_features'], results_df['auc'], 's-', linewidth=2, markersize=8, color='orange')
ax.axhline(0.8914, color='r', linestyle='--', label='Baseline: 0.8914')
ax.set_xlabel('Number of Features', fontsize=11)
ax.set_ylabel('AUC Score', fontsize=11)
ax.set_title('AUC vs Feature Count', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# 3. FPR vs 피처 수
ax = axes[0, 2]
ax.plot(results_df['n_features'], results_df['fpr'] * 100, '^-', linewidth=2, markersize=8, color='red')
ax.axhline(25, color='gray', linestyle=':', label='Target: 25%')
ax.set_xlabel('Number of Features', fontsize=11)
ax.set_ylabel('False Positive Rate (%)', fontsize=11)
ax.set_title('FPR vs Feature Count', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# 4. 학습 시간 vs 피처 수
ax = axes[1, 0]
ax.bar(results_df['n_features'].astype(str), results_df['wall_time'],
       alpha=0.7, edgecolor='black', color='steelblue')
ax.set_xlabel('Number of Features', fontsize=11)
ax.set_ylabel('Training Time (seconds)', fontsize=11)
ax.set_title('Training Time vs Feature Count', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')

# 5. Precision vs Recall
ax = axes[1, 1]
for idx, row in results_df.iterrows():
    ax.scatter(row['recall'], row['precision'], s=150, alpha=0.7,
               label=f"{row['n_features']:.0f} features")
ax.set_xlabel('Recall', fontsize=11)
ax.set_ylabel('Precision', fontsize=11)
ax.set_title('Precision vs Recall Trade-off', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# 6. 종합 성능 비교 (막대 그래프)
ax = axes[1, 2]
x = np.arange(len(results_df))
width = 0.35

f1_normalized = (results_df['f1'] - results_df['f1'].min()) / (results_df['f1'].max() - results_df['f1'].min())
speed_normalized = 1 - (results_df['wall_time'] - results_df['wall_time'].min()) / (results_df['wall_time'].max() - results_df['wall_time'].min())

ax.bar(x - width/2, f1_normalized, width, label='F1 (Normalized)', alpha=0.8)
ax.bar(x + width/2, speed_normalized, width, label='Speed (Normalized)', alpha=0.8)

ax.set_xlabel('Feature Count', fontsize=11)
ax.set_ylabel('Normalized Score', fontsize=11)
ax.set_title('Performance vs Speed (Normalized)', fontsize=12, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels([f"{int(n)}" for n in results_df['n_features']])
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('feature_count_optimization.png', dpi=300, bbox_inches='tight')
plt.close()
print(f"\n✓ 시각화 저장: feature_count_optimization.png")

# ============================================================
# 5.6 최종 권장사항
# ============================================================
print(f"\n{'='*70}")
print("최종 권장사항")
print(f"{'='*70}")

# 상황별 권장
print(f"\n[상황별 권장 피처 수]")

print(f"\n1. 최고 정확도 우선")
print(f"   피처 수: {best_f1_row['n_features']:.0f}개")
print(f"   F1 Score: {best_f1_row['f1']:.4f}")
print(f"   FPR: {best_f1_row['fpr']*100:.2f}%")
print(f"   적용: 경쟁, 논문, 정확도 중시")

print(f"\n2. 균형 (권장) ✓")
print(f"   피처 수: {best_tradeoff_row['n_features']:.0f}개")
print(f"   F1 Score: {best_tradeoff_row['f1']:.4f}")
print(f"   학습 시간: {best_tradeoff_row['wall_time']:.1f}초")
print(f"   적용: 일반적인 사용, 프로덕션")

print(f"\n3. 속도 우선")
fastest_idx = results_df['wall_time'].idxmin()
fastest_row = results_df.iloc[fastest_idx]
print(f"   피처 수: {fastest_row['n_features']:.0f}개")
print(f"   F1 Score: {fastest_row['f1']:.4f}")
print(f"   학습 시간: {fastest_row['wall_time']:.1f}초")
print(f"   적용: 실시간 추론, 리소스 제약")

# CSV 저장
results_df.to_csv('feature_count_optimization_results.csv', index=False)
print(f"\n✓ 결과 저장: feature_count_optimization_results.csv")

print(f"\n{'='*70}")
print("✓ 단계 5 완료 - 최적 피처 수 탐색 완료")
print(f"{'='*70}")

# 전역 변수 저장
globals().update({
    'optimization_results': all_results,
    'best_n_features': int(best_f1_row['n_features']),
    'best_f1_score': best_f1_row['f1']
})


단계 5: 최적 피처 수 탐색 (Grid Search)

[목표]
  1. ✓ 다양한 피처 수 (100~300) 실험
  2. ✓ 성능 vs 효율성 Trade-off 분석
  3. ✓ 최적 피처 수 결정
  4. ✓ 최종 모델 선택

데이터 준비

[실험 설정]
  테스트할 피처 수: [100, 150, 200, 250, 300]
  5-Fold Cross-Validation
  3-Model Ensemble (LGBM 25%, XGB 50%, CAT 25%)

Grid Search 실행

[100개 피처]
    ✓ 5-Fold 완료
  F1 Score:  0.8184
  AUC:       0.8805
  FPR:       0.2488 (24.88%)
  Threshold: 0.415
  Low Conf:  1118개 (정확도 0.5608)
  학습 시간: 99.7초

[150개 피처]
    ✓ 5-Fold 완료
  F1 Score:  0.8243
  AUC:       0.8857
  FPR:       0.2493 (24.93%)
  Threshold: 0.405
  Low Conf:  1049개 (정확도 0.5577)
  학습 시간: 123.2초

[200개 피처]
    ✓ 5-Fold 완료
  F1 Score:  0.8275
  AUC:       0.8890
  FPR:       0.2488 (24.88%)
  Threshold: 0.390
  Low Conf:  983개 (정확도 0.5738)
  학습 시간: 163.2초

[250개 피처]
    ✓ 5-Fold 완료
  F1 Score:  0.8294
  AUC:       0.8909
  FPR:       0.2427 (24.27%)
  Threshold: 0.400
  Low Conf:  1017개 (정확도 0.5693)
  학습 시간: 192.1초

[300개 피처]
    ✓ 5-Fold 완료
  F1 Score:  0.8303
  AUC:       0.8925
  FPR: 

In [13]:
# ============================================================
# 단계 6: 최종 모델 구성 및 Test 예측 (Top 300 피처)
# ============================================================

import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, roc_auc_score, confusion_matrix
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

print("=" * 70)
print("단계 6: 최종 모델 구성 및 Test 예측 (Top 300)")
print("=" * 70)
print("\n[최종 설정]")
print("  피처 수: 300개 (최고 성능)")
print("  Threshold: 0.390 (5단계 결과)")
print("  Ensemble: LGBM(25%) + XGB(50%) + CAT(25%)")
print("  목표 F1: 0.8303, AUC: 0.8925")

# ============================================================
# 6.1 Top 300 피처 선택 및 데이터 준비
# ============================================================
print(f"\n{'='*70}")
print("데이터 준비")
print(f"{'='*70}")

# Feature importance 로드 및 Top 300 선택
importance_df = pd.read_csv('feature_importance_ensemble_cv.csv')
selected_features = importance_df.head(300)['feature'].tolist()

print(f"\n[Top 300 피처]")
print(f"  총 피처: 300개")

# 타입별 분포
fp_cols = [f for f in selected_features if f.startswith(('ecfp_', 'fcfp_', 'ptfp_'))]
desc_cols = [f for f in selected_features if f in ['MolWt', 'clogp', 'sa_score', 'qed']]

ecfp_count = len([f for f in fp_cols if f.startswith('ecfp_')])
fcfp_count = len([f for f in fp_cols if f.startswith('fcfp_')])
ptfp_count = len([f for f in fp_cols if f.startswith('ptfp_')])

print(f"  - Descriptor: {len(desc_cols)}개")
print(f"  - Fingerprint: {len(fp_cols)}개")
print(f"    · ECFP: {ecfp_count}개")
print(f"    · FCFP: {fcfp_count}개")
print(f"    · PTFP: {ptfp_count}개")

# 데이터 로드
df_train = pd.read_csv('train.csv')
X_train = df_train[selected_features]
y_train = df_train['label'].astype(int)

print(f"\n[Train 데이터]")
print(f"  Shape: {X_train.shape}")
print(f"  Label 분포: Class 0 = {sum(y_train==0)}, Class 1 = {sum(y_train==1)}")

# Test 데이터 로드
try:
    df_test = pd.read_csv('predict_input.csv')
    X_test = df_test[selected_features]
    print(f"\n[Test 데이터]")
    print(f"  Shape: {X_test.shape}")
    test_available = True
except FileNotFoundError:
    print(f"\n⚠️  Test 데이터(predict_input.csv) 없음 - 학습만 진행")
    test_available = False

# ============================================================
# 6.2 전처리 파이프라인
# ============================================================
preprocessor = ColumnTransformer(
    transformers=[
        ('fp', SimpleImputer(strategy='constant', fill_value=0), fp_cols),
        ('desc', Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), desc_cols)
    ],
    remainder='drop'
)

# ============================================================
# 6.3 교차검증으로 최종 모델 학습 및 검증
# ============================================================
print(f"\n{'='*70}")
print("최종 모델 학습 및 검증 (5-Fold CV)")
print(f"{'='*70}")

RANDOM_STATE = 42
OPTIMAL_THRESHOLD = 0.390  # 5단계 결과

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

# 결과 저장
final_results = {
    'lgbm': {'models': [], 'oof_probabilities': np.zeros(len(X_train))},
    'xgb': {'models': [], 'oof_probabilities': np.zeros(len(X_train))},
    'catboost': {'models': [], 'oof_probabilities': np.zeros(len(X_train))},
    'ensemble': {'oof_probabilities': np.zeros(len(X_train)),
                 'oof_predictions': np.zeros(len(X_train))},
    'fold_details': []
}

for fold, (tr_idx, va_idx) in enumerate(skf.split(X_train, y_train), 1):
    print(f"\n{'─'*70}")
    print(f"📊 Fold {fold}/5")
    print(f"{'─'*70}")

    X_tr, X_va = X_train.iloc[tr_idx], X_train.iloc[va_idx]
    y_tr, y_va = y_train.iloc[tr_idx], y_train.iloc[va_idx]

    # 전처리
    Xt_tr = preprocessor.fit_transform(X_tr)
    Xt_va = preprocessor.transform(X_va)

    print(f"  학습: {Xt_tr.shape}, 검증: {Xt_va.shape}")

    # ========================================
    # LightGBM
    # ========================================
    print(f"  [1/3] LightGBM...", end=' ')
    lgbm_model = LGBMClassifier(
        n_estimators=1000,
        learning_rate=0.03,
        max_depth=8,
        num_leaves=63,
        min_child_samples=30,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.3,
        reg_lambda=0.3,
        class_weight={0: 1.5, 1: 1.0},
        random_state=RANDOM_STATE,
        n_jobs=-1,
        verbose=-1
    )

    lgbm_model.fit(
        Xt_tr, y_tr,
        eval_set=[(Xt_va, y_va)],
        callbacks=[lgb.early_stopping(stopping_rounds=100, verbose=False)]
    )

    lgbm_proba = lgbm_model.predict_proba(Xt_va)[:, 1]
    lgbm_pred = (lgbm_proba >= OPTIMAL_THRESHOLD).astype(int)
    lgbm_f1 = f1_score(y_va, lgbm_pred)

    final_results['lgbm']['models'].append(lgbm_model)
    final_results['lgbm']['oof_probabilities'][va_idx] = lgbm_proba

    print(f"F1: {lgbm_f1:.4f}, Iter: {lgbm_model.best_iteration_}")

    # ========================================
    # XGBoost
    # ========================================
    print(f"  [2/3] XGBoost...", end=' ')
    xgb_model = XGBClassifier(
        n_estimators=1000,
        learning_rate=0.03,
        max_depth=7,
        min_child_weight=3,
        subsample=0.8,
        colsample_bytree=0.8,
        gamma=0.1,
        reg_alpha=0.3,
        reg_lambda=0.3,
        scale_pos_weight=0.67,
        random_state=RANDOM_STATE,
        n_jobs=-1,
        early_stopping_rounds=100,
        eval_metric='logloss',
        verbosity=0
    )

    xgb_model.fit(
        Xt_tr, y_tr,
        eval_set=[(Xt_va, y_va)],
        verbose=False
    )

    xgb_proba = xgb_model.predict_proba(Xt_va)[:, 1]
    xgb_pred = (xgb_proba >= OPTIMAL_THRESHOLD).astype(int)
    xgb_f1 = f1_score(y_va, xgb_pred)

    final_results['xgb']['models'].append(xgb_model)
    final_results['xgb']['oof_probabilities'][va_idx] = xgb_proba

    print(f"F1: {xgb_f1:.4f}, Iter: {xgb_model.best_iteration}")

    # ========================================
    # CatBoost
    # ========================================
    print(f"  [3/3] CatBoost...", end=' ')
    cat_model = CatBoostClassifier(
        iterations=1000,
        learning_rate=0.03,
        depth=7,
        l2_leaf_reg=3,
        class_weights=[1.5, 1.0],
        random_seed=RANDOM_STATE,
        verbose=0,
        early_stopping_rounds=100
    )

    cat_model.fit(
        Xt_tr, y_tr,
        eval_set=(Xt_va, y_va),
        verbose=False
    )

    cat_proba = cat_model.predict_proba(Xt_va)[:, 1]
    cat_pred = (cat_proba >= OPTIMAL_THRESHOLD).astype(int)
    cat_f1 = f1_score(y_va, cat_pred)

    final_results['catboost']['models'].append(cat_model)
    final_results['catboost']['oof_probabilities'][va_idx] = cat_proba

    print(f"F1: {cat_f1:.4f}, Iter: {cat_model.best_iteration_}")

    # ========================================
    # Ensemble
    # ========================================
    ensemble_proba = (
        0.25 * lgbm_proba +
        0.50 * xgb_proba +
        0.25 * cat_proba
    )
    ensemble_pred = (ensemble_proba >= OPTIMAL_THRESHOLD).astype(int)

    final_results['ensemble']['oof_probabilities'][va_idx] = ensemble_proba
    final_results['ensemble']['oof_predictions'][va_idx] = ensemble_pred

    ensemble_f1 = f1_score(y_va, ensemble_pred)
    ensemble_auc = roc_auc_score(y_va, ensemble_proba)

    cm = confusion_matrix(y_va, ensemble_pred)
    tn, fp, fn, tp = cm.ravel()
    fpr = fp / (fp + tn)

    print(f"\n  [Ensemble] F1: {ensemble_f1:.4f}, AUC: {ensemble_auc:.4f}, FPR: {fpr:.4f}")

    final_results['fold_details'].append({
        'fold': fold,
        'lgbm_f1': lgbm_f1,
        'xgb_f1': xgb_f1,
        'cat_f1': cat_f1,
        'ensemble_f1': ensemble_f1,
        'ensemble_auc': ensemble_auc,
        'fpr': fpr
    })

# ============================================================
# 6.4 최종 검증 성능
# ============================================================
print(f"\n{'='*70}")
print("최종 검증 성능 (OOF)")
print(f"{'='*70}")

# OOF 성능
oof_f1 = f1_score(y_train, final_results['ensemble']['oof_predictions'])
oof_auc = roc_auc_score(y_train, final_results['ensemble']['oof_probabilities'])

oof_cm = confusion_matrix(y_train, final_results['ensemble']['oof_predictions'])
tn, fp, fn, tp = oof_cm.ravel()
oof_fpr = fp / (fp + tn)
oof_precision = tp / (tp + fp)
oof_recall = tp / (tp + fn)

print(f"\n[Ensemble OOF 성능]")
print(f"  F1 Score:  {oof_f1:.4f}")
print(f"  AUC Score: {oof_auc:.4f}")
print(f"  Precision: {oof_precision:.4f}")
print(f"  Recall:    {oof_recall:.4f}")
print(f"  FPR:       {oof_fpr:.4f} ({oof_fpr*100:.2f}%)")

print(f"\n[OOF 혼동 행렬]")
print(f"              예측: 0    예측: 1")
print(f"  실제: 0  |   {oof_cm[0,0]:4d}      {oof_cm[0,1]:4d}")
print(f"  실제: 1  |   {oof_cm[1,0]:4d}      {oof_cm[1,1]:4d}")

# Low Confidence
confidence = np.abs(final_results['ensemble']['oof_probabilities'] - 0.5)
low_conf_mask = confidence < 0.1
n_low_conf = low_conf_mask.sum()
low_conf_acc = (
    final_results['ensemble']['oof_predictions'][low_conf_mask] == y_train[low_conf_mask]
).mean() if n_low_conf > 0 else 0

print(f"\n[Low Confidence 샘플]")
print(f"  개수: {n_low_conf}개 ({n_low_conf/len(y_train)*100:.2f}%)")
print(f"  정확도: {low_conf_acc:.4f}")

# ============================================================
# 6.5 Test 데이터 예측
# ============================================================
if test_available:
    print(f"\n{'='*70}")
    print("Test 데이터 예측")
    print(f"{'='*70}")

    # 전체 Train 데이터로 최종 전처리기 학습
    Xt_train_full = preprocessor.fit_transform(X_train)
    Xt_test = preprocessor.transform(X_test)

    print(f"\n  Test 데이터 shape: {Xt_test.shape}")

    # 각 Fold 모델로 예측 후 앙상블
    test_predictions = {
        'lgbm': np.zeros((len(X_test), 5)),
        'xgb': np.zeros((len(X_test), 5)),
        'catboost': np.zeros((len(X_test), 5))
    }

    for fold in range(5):
        print(f"\r  Fold {fold+1}/5 모델로 예측 중...", end='')

        test_predictions['lgbm'][:, fold] = final_results['lgbm']['models'][fold].predict_proba(Xt_test)[:, 1]
        test_predictions['xgb'][:, fold] = final_results['xgb']['models'][fold].predict_proba(Xt_test)[:, 1]
        test_predictions['catboost'][:, fold] = final_results['catboost']['models'][fold].predict_proba(Xt_test)[:, 1]

    print(f"\r  ✓ 5-Fold 예측 완료")

    # 평균 확률
    lgbm_proba_test = test_predictions['lgbm'].mean(axis=1)
    xgb_proba_test = test_predictions['xgb'].mean(axis=1)
    cat_proba_test = test_predictions['catboost'].mean(axis=1)

    # Ensemble
    ensemble_proba_test = (
        0.25 * lgbm_proba_test +
        0.50 * xgb_proba_test +
        0.25 * cat_proba_test
    )

    # 최종 예측 (Threshold 적용)
    ensemble_pred_test = (ensemble_proba_test >= OPTIMAL_THRESHOLD).astype(int)

    # Confidence 계산
    confidence_test = np.abs(ensemble_proba_test - 0.5)

    print(f"\n[Test 예측 결과]")
    print(f"  예측 Class 0: {sum(ensemble_pred_test == 0)}개")
    print(f"  예측 Class 1: {sum(ensemble_pred_test == 1)}개")
    print(f"  평균 Confidence: {confidence_test.mean():.4f}")
    print(f"  Low Confidence (<0.1): {sum(confidence_test < 0.1)}개 ({sum(confidence_test < 0.1)/len(confidence_test)*100:.2f}%)")

    # ============================================================
    # 6.6 Submission 파일 생성
    # ============================================================
    print(f"\n{'='*70}")
    print("Submission 파일 생성")
    print(f"{'='*70}")

    # 기본 Submission
    submission = pd.DataFrame({
        'id': range(len(ensemble_pred_test)),
        'label': ensemble_pred_test
    })

    submission.to_csv('submission_final_top300.csv', index=False)
    print(f"\n✓ 기본 제출 파일: submission_final_top300.csv")

    # 상세 Submission (확률 포함)
    submission_detailed = pd.DataFrame({
        'id': range(len(ensemble_pred_test)),
        'label': ensemble_pred_test,
        'probability': ensemble_proba_test,
        'confidence': confidence_test,
        'lgbm_proba': lgbm_proba_test,
        'xgb_proba': xgb_proba_test,
        'catboost_proba': cat_proba_test
    })

    submission_detailed.to_csv('submission_detailed_final_top300.csv', index=False)
    print(f"✓ 상세 제출 파일: submission_detailed_final_top300.csv")

    # 통계
    print(f"\n[제출 파일 통계]")
    print(f"  전체 샘플: {len(submission)}개")
    print(f"  Class 0 (독성): {sum(submission['label'] == 0)}개 ({sum(submission['label'] == 0)/len(submission)*100:.2f}%)")
    print(f"  Class 1 (무독성): {sum(submission['label'] == 1)}개 ({sum(submission['label'] == 1)/len(submission)*100:.2f}%)")

# ============================================================
# 6.7 최종 모델 저장
# ============================================================
print(f"\n{'='*70}")
print("최종 모델 저장")
print(f"{'='*70}")

import pickle

# 모델 및 설정 저장
final_model_package = {
    'lgbm_models': final_results['lgbm']['models'],
    'xgb_models': final_results['xgb']['models'],
    'catboost_models': final_results['catboost']['models'],
    'selected_features': selected_features,
    'preprocessor': preprocessor,
    'optimal_threshold': OPTIMAL_THRESHOLD,
    'oof_f1': oof_f1,
    'oof_auc': oof_auc,
    'oof_fpr': oof_fpr
}

with open('final_model_top300.pkl', 'wb') as f:
    pickle.dump(final_model_package, f)

print(f"\n✓ 모델 저장: final_model_top300.pkl")
print(f"  - 5-Fold × 3-Model = 15개 모델")
print(f"  - Top 300 피처 리스트")
print(f"  - 전처리 파이프라인")
print(f"  - 최적 Threshold: {OPTIMAL_THRESHOLD}")

# 피처 리스트 별도 저장
pd.DataFrame({'feature': selected_features}).to_csv('selected_features_top300.csv', index=False)
print(f"✓ 피처 리스트: selected_features_top300.csv")

# ============================================================
# 6.8 최종 성능 리포트
# ============================================================
print(f"\n{'='*70}")
print("최종 성능 리포트")
print(f"{'='*70}")

print(f"\n[모델 사양]")
print(f"  피처 수: 300개 (압축률 90.2%)")
print(f"  Threshold: {OPTIMAL_THRESHOLD}")
print(f"  Ensemble: LGBM(25%) + XGB(50%) + CAT(25%)")

print(f"\n[검증 성능 (5-Fold CV OOF)]")
print(f"  F1 Score:  {oof_f1:.4f}")
print(f"  AUC Score: {oof_auc:.4f}")
print(f"  Precision: {oof_precision:.4f}")
print(f"  Recall:    {oof_recall:.4f}")
print(f"  FPR:       {oof_fpr*100:.2f}%")

print(f"\n[Fold별 성능]")
print(f"{'Fold':<6} {'LGBM F1':<10} {'XGB F1':<10} {'CAT F1':<10} {'Ensemble':<10} {'FPR':<8}")
print(f"{'-'*60}")
for detail in final_results['fold_details']:
    print(f"{detail['fold']:<6} {detail['lgbm_f1']:<10.4f} {detail['xgb_f1']:<10.4f} "
          f"{detail['cat_f1']:<10.4f} {detail['ensemble_f1']:<10.4f} {detail['fpr']*100:<8.2f}%")

print(f"\n[2단계 (3076개 피처) 대비]")
baseline_f1 = 0.8317
baseline_fpr = 0.2514
print(f"  F1 Score:  {baseline_f1:.4f} → {oof_f1:.4f} ({(oof_f1-baseline_f1)*100:+.2f}%p)")
print(f"  FPR:       {baseline_fpr*100:.2f}% → {oof_fpr*100:.2f}% ({(oof_fpr-baseline_fpr)*100:+.2f}%p)")
print(f"  피처 수:    3076개 → 300개 (-90.2%)")

if test_available:
    print(f"\n[Test 예측]")
    print(f"  예측 완료: {len(ensemble_pred_test)}개")
    print(f"  Class 1 비율: {sum(ensemble_pred_test == 1)/len(ensemble_pred_test)*100:.2f}%")
    print(f"  제출 파일: submission_final_top300.csv")

print(f"\n{'='*70}")
print("✓ 단계 6 완료 - 최종 모델 구성 완료")
print(f"{'='*70}")
print(f"\n[다음 단계]")
if test_available:
    print(f"  1. submission_final_top300.csv 제출")
    print(f"  2. 성능 확인 후 피드백")
    print(f"  3. 필요 시 Threshold 재조정")
else:
    print(f"  1. predict_input.csv 준비")
    print(f"  2. 코드 재실행")


단계 6: 최종 모델 구성 및 Test 예측 (Top 300)

[최종 설정]
  피처 수: 300개 (최고 성능)
  Threshold: 0.390 (5단계 결과)
  Ensemble: LGBM(25%) + XGB(50%) + CAT(25%)
  목표 F1: 0.8303, AUC: 0.8925

데이터 준비

[Top 300 피처]
  총 피처: 300개
  - Descriptor: 4개
  - Fingerprint: 296개
    · ECFP: 92개
    · FCFP: 79개
    · PTFP: 125개

[Train 데이터]
  Shape: (8349, 300)
  Label 분포: Class 0 = 3807, Class 1 = 4542

[Test 데이터]
  Shape: (927, 300)

최종 모델 학습 및 검증 (5-Fold CV)

──────────────────────────────────────────────────────────────────────
📊 Fold 1/5
──────────────────────────────────────────────────────────────────────
  학습: (6679, 300), 검증: (1670, 300)
  [1/3] LightGBM... F1: 0.8388, Iter: 655
  [2/3] XGBoost... F1: 0.8507, Iter: 956
  [3/3] CatBoost... F1: 0.8385, Iter: 994

  [Ensemble] F1: 0.8464, AUC: 0.9065, FPR: 0.2313

──────────────────────────────────────────────────────────────────────
📊 Fold 2/5
──────────────────────────────────────────────────────────────────────
  학습: (6679, 300), 검증: (1670, 300)
  [1/3] LightGBM...

In [14]:
# ============================================================
# Submission 파일 형식 변환 (SMILES + output)
# ============================================================

import pandas as pd

print("=" * 70)
print("Submission 파일 형식 변환")
print("=" * 70)

# ============================================================
# 1. 기존 submission 파일 로드
# ============================================================
print("\n[1단계] 파일 로드")

# 6단계에서 생성한 submission 파일
submission = pd.read_csv('submission_final_top300.csv')
print(f"  기존 submission: {submission.shape}")
print(f"  컬럼: {submission.columns.tolist()}")

# Test 데이터에서 SMILES 정보 가져오기
test_data = pd.read_csv('predict_input.csv')

# SMILES 컬럼 확인
if 'SMILES' in test_data.columns:
    smiles_col = 'SMILES'
elif 'smiles' in test_data.columns:
    smiles_col = 'smiles'
else:
    # 첫 번째 컬럼이 SMILES일 가능성
    smiles_col = test_data.columns[0]
    print(f"  ⚠️  'SMILES' 컬럼 없음, '{smiles_col}' 사용")

print(f"  Test 데이터: {test_data.shape}")
print(f"  SMILES 컬럼: {smiles_col}")

# ============================================================
# 2. 형식 변환
# ============================================================
print(f"\n[2단계] 형식 변환")

# 새로운 submission 형식 생성
submission_final = pd.DataFrame({
    'SMILES': test_data[smiles_col],
    'output': submission['label']
})

print(f"  변환 완료: {submission_final.shape}")
print(f"  컬럼: {submission_final.columns.tolist()}")

# ============================================================
# 3. 통계 확인
# ============================================================
print(f"\n[3단계] 통계 확인")

print(f"\n  예측 분포:")
print(f"    output=0 (독성): {sum(submission_final['output'] == 0)}개 ({sum(submission_final['output'] == 0)/len(submission_final)*100:.2f}%)")
print(f"    output=1 (무독성): {sum(submission_final['output'] == 1)}개 ({sum(submission_final['output'] == 1)/len(submission_final)*100:.2f}%)")

# 샘플 확인
print(f"\n  처음 10개 샘플:")
print(submission_final.head(10).to_string(index=False))

# ============================================================
# 4. 파일 저장
# ============================================================
print(f"\n[4단계] 파일 저장")

# 최종 제출 파일
submission_final.to_csv('submission_final_format.csv', index=False)
print(f"  ✓ 저장 완료: submission_final_format.csv")

# 검증: 파일 다시 읽어서 확인
verify = pd.read_csv('submission_final_format.csv')
print(f"\n[검증]")
print(f"  파일 크기: {len(verify)}행 × {len(verify.columns)}열")
print(f"  컬럼: {verify.columns.tolist()}")
print(f"  첫 5행:")
print(verify.head().to_string(index=False))

# ============================================================
# 5. 예상 제출 형식 매칭 확인
# ============================================================
print(f"\n{'='*70}")
print("제출 형식 최종 확인")
print(f"{'='*70}")

print(f"\n[요구 형식 (사진 기준)]")
print(f"  컬럼 1: SMILES (분자 구조)")
print(f"  컬럼 2: output (예측값 0 또는 1)")

print(f"\n[생성된 파일]")
print(f"  컬럼 1: {verify.columns[0]} ✓")
print(f"  컬럼 2: {verify.columns[1]} ✓")

if verify.columns[0] == 'SMILES' and verify.columns[1] == 'output':
    print(f"\n✓✓✓ 형식 매칭 완료 - 제출 가능!")
else:
    print(f"\n⚠️  컬럼명 확인 필요")

print(f"\n{'='*70}")
print("✓ 변환 완료 - submission_final_format.csv 제출하세요!")
print(f"{'='*70}")


Submission 파일 형식 변환

[1단계] 파일 로드
  기존 submission: (927, 2)
  컬럼: ['id', 'label']
  Test 데이터: (927, 3077)
  SMILES 컬럼: SMILES

[2단계] 형식 변환
  변환 완료: (927, 2)
  컬럼: ['SMILES', 'output']

[3단계] 통계 확인

  예측 분포:
    output=0 (독성): 378개 (40.78%)
    output=1 (무독성): 549개 (59.22%)

  처음 10개 샘플:
                                          SMILES  output
                           OC(=O)c1cc2sccc2[nH]1       1
                     [O-][n+]1onc(c2ccccc2)c1C#N       1
                      CN1C(=O)N(C)c2ncn(C)c2C1=O       1
                          Clc1cccc(c1)C2CNCC=CC2       1
                      CCN(CC)CC(=O)Nc1c(C)cccc1C       1
                      CCN(CC)CCNC(=O)c1ccc(N)cc1       1
NC[C@H]1C[C@@H]1c2cc(Cl)ccc2OCC=C.OC(=O)C(F)(F)F       1
                         Clc1ccc(cc1Cl)C2CCCCNC2       1
                NC(=O)C1CCCc2c1[nH]c3ccc(Cl)cc23       1
        O=C1NOC(=C1)[C@H]2CCN[C@@H](Cc3ccccc3)C2       1

[4단계] 파일 저장
  ✓ 저장 완료: submission_final_format.csv

[검증]
  파일 크기: 927행 × 2열
  컬럼: ['S